In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Working with Paper 1
* This Paper does Prediction on BSE Index data
* The date of their data ranges from 3rd April 1979 to 13th January 2022
* It uses MAE metric
* The best result quoted by them is MAE:402.140232
* Link to paper:-https://scholar.google.com/citations?view_op=view_citation&hl=en&user=yO6rbN8AAAAJ&citation_for_view=yO6rbN8AAAAJ:ULOm3_A8WrAC

In [ ]:
data=pd.read_csv("/kaggle/input/personal-dataset/sensex_paper3.csv")
#data=data[::-1]
data.reset_index(drop=True, inplace=True)
data.nunique()
data.head(),data.dtypes,data.isnull().sum()


In [ ]:
import pandas as pd
import numpy as np

# Load CSV
df = pd.read_csv("/kaggle/input/personal-dataset/sensex_paper3.csv")  # Replace with actual filename

# Strip spaces and convert Change % properly
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Convert numeric columns (except 'Vol.') to float
numeric_cols = ["Close", "Open", "High", "Low"]
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col].str.replace(',', '', regex=True), errors='coerce')
def clean_change(x):
    if isinstance(x, str):
        # Remove the '%' symbol (only at the end, if present)
        if x.endswith('%'):
            x = x[:-1]
        return x.strip()
    return x

df["Change %"] = df["Change %"].apply(clean_change)
df["Change %"] = pd.to_numeric(df["Change %"], errors='coerce')


# Convert 'Vol.' column while keeping important data
def convert_volume(val):
    if pd.isna(val) or val == '-':  # Handle NaN or '-' cases
        return np.nan
    val = val.strip().replace(',', '')  # Remove spaces & commas
    if val[-1] == 'K': return float(val[:-1]) * 1e3
    if val[-1] == 'M': return float(val[:-1]) * 1e6
    if val[-1] == 'B': return float(val[:-1]) * 1e9
    return float(val)

df["Vol."] = df["Vol."].apply(convert_volume)

# Drop Date column
df.drop(columns=["Date","Vol.","Change %"], inplace=True)

# Forward fill missing 'Vol.' values to retain important data
#df["Vol."].fillna(method='ffill', inplace=True)

# Drop rows where 'Close' is still NaN (essential for LSTM training)
df.dropna(subset=["Close"], inplace=True)

# Final check
print(df.info())  # Ensure all columns are numeric
print(df.head())  # Verify cleaned data
data=df
data.isnull().sum(),data.dtypes

In [ ]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, explained_variance_score
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, Bidirectional, LSTM, Lambda, Concatenate, Add, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# ---------------------------
# 1. Set Seed for Reproducibility
# ---------------------------
def set_random_seed(seed=20):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.config.experimental.enable_op_determinism()

set_random_seed(20)

# ---------------------------
# 2. Load & Preprocess Data
# ---------------------------
# For demonstration, we use a basic univariate "Close" series.
# Ensure that your DataFrame 'data' contains at least a "Close" column.
df = data[["Close"]].copy()
scaler = MinMaxScaler()
df["Close"] = scaler.fit_transform(df)

# ---------------------------
# 3. Create 15-day Sequences
# ---------------------------
def create_sequences(data, time_steps=15):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i : i + time_steps])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

X, y = create_sequences(df["Close"].values, time_steps=15)

# Split data: 70% train, 10% validation, 20% test.
train_size = int(len(X) * 0.7)
val_size = int(len(X) * 0.1)
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val     = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test   = X[train_size+val_size:], y[train_size+val_size:]

# Reshape each sample to (time_steps, 1)
X_train = X_train.reshape(-1, 15, 1)
X_val   = X_val.reshape(-1, 15, 1)
X_test  = X_test.reshape(-1, 15, 1)

# ---------------------------
# 4. Build Multi-Scale Hybrid Model
# ---------------------------
from tensorflow.keras.layers import Multiply, Activation

def build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, 1))

    # *Branch 1: Full 15-Day Analysis (Long-Term Trends)*
    full_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(inputs)
    full_branch = Dense(64, activation='relu')(full_branch)
    long_term_return = Dense(1, activation='tanh')(full_branch)
    long_term_return = Lambda(lambda x: x * scale_factor)(long_term_return)

    # *Branch 2: Recent 7-Day Analysis (Mid-Term Trends)*
    recent_input = Lambda(lambda x: x[:, -7:, :])(inputs)  # Take the last 7 days
    recent_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(recent_input)
    recent_branch = Dense(64, activation='relu')(recent_branch)
    mid_term_return = Dense(1, activation='tanh')(recent_branch)
    mid_term_return = Lambda(lambda x: x * scale_factor)(mid_term_return)

    # *Branch 3: Very Recent 1-Day Analysis (Short-Term Trends)*
    very_recent_input = Lambda(lambda x: x[:, -1:, :])(inputs)  # Take the last day's price
    very_recent_branch = Flatten()(very_recent_input)
    very_recent_branch = Dense(32, activation='relu')(very_recent_branch)
    short_term_return = Dense(1, activation='tanh')(very_recent_branch)
    short_term_return = Lambda(lambda x: x * recent_scale)(short_term_return)

    # *Gating Mechanism (Forget Gate)*
    gate_7 = Dense(1, activation='sigmoid')(mid_term_return)  # Gate for 7-day impact
    gate_1 = Dense(1, activation='sigmoid')(short_term_return)  # Gate for 1-day impact

    gated_mid_term_return = Multiply()([mid_term_return, gate_7])  # 7-day result scaled by gate
    gated_short_term_return = Multiply()([short_term_return, gate_1])  # 1-day result scaled by gate

    # *Combine Long-Term, Gated Mid-Term, and Gated Short-Term Predictions*
    fused_returns = Add()([long_term_return, gated_mid_term_return, gated_short_term_return])
    fused_returns = Dense(32, activation='relu')(fused_returns)
    final_return = Dense(1, activation='tanh')(fused_returns)
    final_return = Lambda(lambda x: x * scale_factor)(final_return)

    # *Use the last day's price for the final output calculation*
    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, final_return])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

r2 = r2_score(y_test_inv, y_pred_inv)
rmse = mean_squared_error(y_test_inv, y_pred_inv, squared=False)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mape = mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100
evs = explained_variance_score(y_test_inv, y_pred_inv)

print(f"📊 Final Results - R²: {r2:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}, MAPE: {mape:.2f}%, EVS: {evs:.4f}")

plt.figure(figsize=(10,5))
plt.plot(y_test_inv, label='Actual')
plt.plot(y_pred_inv, label='Predicted')
plt.xlabel("Time")
plt.ylabel("Close Price")
plt.legend()
plt.show()

* Now we try to add another parameter,namely the India VIX
* However the VIX index ranges from Only 4th March 2008,so we have to run this model on a subset of the data to check our results
* the data we try our model on thus ranges from 04-03-2008 to 30-04-2021 

In [ ]:
data=pd.read_csv("/kaggle/input/historical-india-stock-market/BSE Sensex 30 Historical Data.csv")
data=data[::-1]
data.reset_index(drop=True, inplace=True)
data.head()
data.nunique()

data.sort_index(axis=1,ascending=True)
data.rename(columns={'Price': 'Close'}, inplace=True)
df = data.copy()  # Ensure we don't modify the original dataset

# Convert financial columns to numeric (remove commas)
for col in ["Close", "Open", "High", "Low"]:
    df[col] = df[col].astype(str).str.replace(",", "").astype(float)

# Function to convert 'Vol.' column
def convert_volume(vol):
    if isinstance(vol, str):  # Ensure it's a string before replacing
        vol = vol.replace(",", "")  # Remove any thousand separators
        if "B" in vol:
            return float(vol.replace("B", "")) * 1_000_000_000
        elif "M" in vol:
            return float(vol.replace("M", "")) * 1_000_000
        elif "K" in vol:
            return float(vol.replace("K", "")) * 1_000
    return float(vol)  # Convert directly if already a number

df["Vol."] = df["Vol."].astype(str).apply(convert_volume)

# Convert 'Change %' column (remove '%' and convert to float)
df["Change %"] = df["Change %"].astype(str).str.replace("%", "").astype(float)

# Print final DataFrame
print(df.dtypes)
print(df)

# Assign back to 'data' (if needed)
data = df

#data = pd.merge(data, vix, on='Date', how='inner')
#data.drop(['Date'], axis=1, inplace=True)
data.head()

In [ ]:
vix = pd.read_csv("/kaggle/input/vix-data/vix.csv")
vix.drop("Vol.",axis=1,inplace=True)
vix["Change %"] = vix["Change %"].str.replace("%", "").astype(float)
vix.head(),vix.dtypes

In [ ]:
data = pd.merge(data, vix, on='Date', how='inner')
data.drop("Date",axis=1,inplace=True)
print(data.head())
print(len(data))


In [ ]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, explained_variance_score
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, Bidirectional, LSTM, Lambda, Concatenate, Add, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# ---------------------------
# 1. Set Seed for Reproducibility
# ---------------------------
def set_random_seed(seed=20):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.config.experimental.enable_op_determinism()

set_random_seed(20)

# ---------------------------
# 2. Load & Preprocess Data
# ---------------------------
# For demonstration, we use a basic univariate "Close" series.
# Ensure that your DataFrame 'data' contains at least a "Close" column.
df = data[["Close"]].copy()
scaler = MinMaxScaler()
df["Close"] = scaler.fit_transform(df)

# ---------------------------
# 3. Create 15-day Sequences
# ---------------------------
def create_sequences(data, time_steps=15):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i : i + time_steps])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

X, y = create_sequences(df["Close"].values, time_steps=15)

# Split data: 70% train, 10% validation, 20% test.
train_size = int(len(X) * 0.7)
val_size = int(len(X) * 0.1)
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val     = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test   = X[train_size+val_size:], y[train_size+val_size:]

# Reshape each sample to (time_steps, 1)
X_train = X_train.reshape(-1, 15, 1)
X_val   = X_val.reshape(-1, 15, 1)
X_test  = X_test.reshape(-1, 15, 1)

# ---------------------------
# 4. Build Multi-Scale Hybrid Model
# ---------------------------
from tensorflow.keras.layers import Multiply, Activation

def build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, 1))

    # *Branch 1: Full 15-Day Analysis (Long-Term Trends)*
    full_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(inputs)
    full_branch = Dense(64, activation='relu')(full_branch)
    long_term_return = Dense(1, activation='tanh')(full_branch)
    long_term_return = Lambda(lambda x: x * scale_factor)(long_term_return)

    # *Branch 2: Recent 7-Day Analysis (Mid-Term Trends)*
    recent_input = Lambda(lambda x: x[:, -7:, :])(inputs)  # Take the last 7 days
    recent_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(recent_input)
    recent_branch = Dense(64, activation='relu')(recent_branch)
    mid_term_return = Dense(1, activation='tanh')(recent_branch)
    mid_term_return = Lambda(lambda x: x * scale_factor)(mid_term_return)

    # *Branch 3: Very Recent 1-Day Analysis (Short-Term Trends)*
    very_recent_input = Lambda(lambda x: x[:, -1:, :])(inputs)  # Take the last day's price
    very_recent_branch = Flatten()(very_recent_input)
    very_recent_branch = Dense(32, activation='relu')(very_recent_branch)
    short_term_return = Dense(1, activation='tanh')(very_recent_branch)
    short_term_return = Lambda(lambda x: x * recent_scale)(short_term_return)

    # *Gating Mechanism (Forget Gate)*
    gate_7 = Dense(1, activation='sigmoid')(mid_term_return)  # Gate for 7-day impact
    gate_1 = Dense(1, activation='sigmoid')(short_term_return)  # Gate for 1-day impact

    gated_mid_term_return = Multiply()([mid_term_return, gate_7])  # 7-day result scaled by gate
    gated_short_term_return = Multiply()([short_term_return, gate_1])  # 1-day result scaled by gate

    # *Combine Long-Term, Gated Mid-Term, and Gated Short-Term Predictions*
    fused_returns = Add()([long_term_return, gated_mid_term_return, gated_short_term_return])
    fused_returns = Dense(32, activation='relu')(fused_returns)
    final_return = Dense(1, activation='tanh')(fused_returns)
    final_return = Lambda(lambda x: x * scale_factor)(final_return)

    # *Use the last day's price for the final output calculation*
    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, final_return])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

r2 = r2_score(y_test_inv, y_pred_inv)
rmse = mean_squared_error(y_test_inv, y_pred_inv, squared=False)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mape = mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100
evs = explained_variance_score(y_test_inv, y_pred_inv)

print(f"📊 Final Results - R²: {r2:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}, MAPE: {mape:.2f}%, EVS: {evs:.4f}")

plt.figure(figsize=(10,5))
plt.plot(y_test_inv, label='Actual')
plt.plot(y_pred_inv, label='Predicted')
plt.xlabel("Time")
plt.ylabel("Close Price")
plt.legend()
plt.show()

## Working with Paper 2
* This paper does prediction on BSE SENSEX data
* Data spans the dates of May 30, 2010, and February 9, 2018
* It uses RMSE and MSE metric
* Their best result is MSE-0.0021 RMSE-0.0438
* Paper link:-https://ieeexplore.ieee.org/document/10397684

In [ ]:
import pandas as pd

# Load the dataset
vix = pd.read_csv("/kaggle/input/vix-data/vix.csv")

# Print initial number of rows
initial_count = len(vix)
print(f"Initial number of rows: {initial_count}")

# Drop the "Vol." column
vix.drop("Vol.", axis=1, inplace=True)

# Convert "Change %" column to float after removing "%"
vix["Change %"] = vix["Change %"].str.replace("%", "").astype(float)

# Convert "Date" column to datetime format
vix["Date"] = pd.to_datetime(vix["Date"])

# Filter data within the specified date range
start_date = "2010-05-30"
end_date = "2018-02-09"
vix = vix[(vix["Date"] >= start_date) & (vix["Date"] <= end_date)]

# Print final number of rows
final_count = len(vix)
print(f"Final number of rows after filtering: {final_count}")

# Display results
vix.head(), vix.dtypes



In [ ]:
import pandas as pd

# Load Sensex dataset
data = pd.read_csv("/kaggle/input/d/abirc8010/historical-india-stock-market/BSE Sensex 30 Historical Data.csv")

# Print initial number of rows
initial_count = len(data)
print(f"Initial number of rows: {initial_count}")

# Reverse and reset index
data = data[::-1].reset_index(drop=True)

# Sort columns alphabetically and rename 'Price' to 'Close'
data.sort_index(axis=1, ascending=True, inplace=True)
data.rename(columns={'Price': 'Close'}, inplace=True)

# Create a copy to avoid modifying the original data
df = data.copy()

# Convert financial columns to numeric
for col in ["Close", "Open", "High", "Low"]:
    df[col] = df[col].astype(str).str.replace(",", "").astype(float)

# Function to convert 'Vol.' column
def convert_volume(vol):
    if isinstance(vol, str):
        vol = vol.replace(",", "")
        if "B" in vol:
            return float(vol.replace("B", "")) * 1_000_000_000
        elif "M" in vol:
            return float(vol.replace("M", "")) * 1_000_000
        elif "K" in vol:
            return float(vol.replace("K", "")) * 1_000
    return float(vol)

df["Vol."] = df["Vol."].astype(str).apply(convert_volume)

# Convert 'Change %' column
df["Change %"] = df["Change %"].astype(str).str.replace("%", "").astype(float)

# Convert 'Date' column to datetime format, handling mixed formats
df["Date"] = pd.to_datetime(df["Date"], dayfirst=True, errors="coerce")

# Filter Sensex data within the specified date range
start_date = "2010-05-30"
end_date = "2018-02-09"
df = df[(df["Date"] >= start_date) & (df["Date"] <= end_date)]

# Print final number of rows
final_count = len(df)
print(f"Final number of rows after filtering: {final_count}")

# Assign back to 'data'
data = df

# Merge with VIX dataset
data = pd.merge(data, vix, on="Date", how="inner")
data.drop(["Date"], axis=1, inplace=True)

# Print number of rows after merging
print(f"Number of rows after merging: {len(data)}")

# Display results
print(data.head())


In [ ]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, explained_variance_score
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, Bidirectional, LSTM, Lambda, Concatenate, Add, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# ---------------------------
# 1. Set Seed for Reproducibility
# ---------------------------
def set_random_seed(seed=20):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.config.experimental.enable_op_determinism()

set_random_seed(20)

# ---------------------------
# 2. Load & Preprocess Data
# ---------------------------
# For demonstration, we use a basic univariate "Close" series.
# Ensure that your DataFrame 'data' contains at least a "Close" column.
df = data[["Close"]].copy()
scaler = MinMaxScaler()
df["Close"] = scaler.fit_transform(df)

# ---------------------------
# 3. Create 15-day Sequences
# ---------------------------
def create_sequences(data, time_steps=15):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i : i + time_steps])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

X, y = create_sequences(df["Close"].values, time_steps=15)

# Split data: 70% train, 10% validation, 20% test.
train_size = int(len(X) * 0.7)
val_size = int(len(X) * 0.1)
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val     = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test   = X[train_size+val_size:], y[train_size+val_size:]

# Reshape each sample to (time_steps, 1)
X_train = X_train.reshape(-1, 15, 1)
X_val   = X_val.reshape(-1, 15, 1)
X_test  = X_test.reshape(-1, 15, 1)

# ---------------------------
# 4. Build Multi-Scale Hybrid Model
# ---------------------------
from tensorflow.keras.layers import Multiply, Activation

def build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, 1))

    # *Branch 1: Full 15-Day Analysis (Long-Term Trends)*
    full_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(inputs)
    full_branch = Dense(64, activation='relu')(full_branch)
    long_term_return = Dense(1, activation='tanh')(full_branch)
    long_term_return = Lambda(lambda x: x * scale_factor)(long_term_return)

    # *Branch 2: Recent 7-Day Analysis (Mid-Term Trends)*
    recent_input = Lambda(lambda x: x[:, -7:, :])(inputs)  # Take the last 7 days
    recent_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(recent_input)
    recent_branch = Dense(64, activation='relu')(recent_branch)
    mid_term_return = Dense(1, activation='tanh')(recent_branch)
    mid_term_return = Lambda(lambda x: x * scale_factor)(mid_term_return)

    # *Branch 3: Very Recent 1-Day Analysis (Short-Term Trends)*
    very_recent_input = Lambda(lambda x: x[:, -1:, :])(inputs)  # Take the last day's price
    very_recent_branch = Flatten()(very_recent_input)
    very_recent_branch = Dense(32, activation='relu')(very_recent_branch)
    short_term_return = Dense(1, activation='tanh')(very_recent_branch)
    short_term_return = Lambda(lambda x: x * recent_scale)(short_term_return)

    # *Gating Mechanism (Forget Gate)*
    gate_7 = Dense(1, activation='sigmoid')(mid_term_return)  # Gate for 7-day impact
    gate_1 = Dense(1, activation='sigmoid')(short_term_return)  # Gate for 1-day impact

    gated_mid_term_return = Multiply()([mid_term_return, gate_7])  # 7-day result scaled by gate
    gated_short_term_return = Multiply()([short_term_return, gate_1])  # 1-day result scaled by gate

    # *Combine Long-Term, Gated Mid-Term, and Gated Short-Term Predictions*
    fused_returns = Add()([long_term_return, gated_mid_term_return, gated_short_term_return])
    fused_returns = Dense(32, activation='relu')(fused_returns)
    final_return = Dense(1, activation='tanh')(fused_returns)
    final_return = Lambda(lambda x: x * scale_factor)(final_return)

    # *Use the last day's price for the final output calculation*
    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, final_return])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

r2 = r2_score(y_test_inv, y_pred_inv)
rmse = mean_squared_error(y_test_inv, y_pred_inv, squared=False)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mape = mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100
evs = explained_variance_score(y_test_inv, y_pred_inv)

print(f"📊 Final Results - R²: {r2:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}, MAPE: {mape:.2f}%, EVS: {evs:.4f}")

plt.figure(figsize=(10,5))
plt.plot(y_test_inv, label='Actual')
plt.plot(y_pred_inv, label='Predicted')
plt.xlabel("Time")
plt.ylabel("Close Price")
plt.legend()
plt.show()

## trying without vix

In [ ]:
import pandas as pd

# Load Sensex dataset
data = pd.read_csv("/kaggle/input/d/abirc8010/historical-india-stock-market/BSE Sensex 30 Historical Data.csv")

# Print initial number of rows
initial_count = len(data)
print(f"Initial number of rows: {initial_count}")

# Reverse and reset index
data = data[::-1].reset_index(drop=True)

# Sort columns alphabetically and rename 'Price' to 'Close'
data.sort_index(axis=1, ascending=True, inplace=True)
data.rename(columns={'Price': 'Close'}, inplace=True)

# Create a copy to avoid modifying the original data
df = data.copy()

# Convert financial columns to numeric
for col in ["Close", "Open", "High", "Low"]:
    df[col] = df[col].astype(str).str.replace(",", "").astype(float)

# Function to convert 'Vol.' column
def convert_volume(vol):
    if isinstance(vol, str):
        vol = vol.replace(",", "")
        if "B" in vol:
            return float(vol.replace("B", "")) * 1_000_000_000
        elif "M" in vol:
            return float(vol.replace("M", "")) * 1_000_000
        elif "K" in vol:
            return float(vol.replace("K", "")) * 1_000
    return float(vol)

df["Vol."] = df["Vol."].astype(str).apply(convert_volume)

# Convert 'Change %' column
df["Change %"] = df["Change %"].astype(str).str.replace("%", "").astype(float)

# Convert 'Date' column to datetime format, handling mixed formats
df["Date"] = pd.to_datetime(df["Date"], dayfirst=True, errors="coerce")

# Filter Sensex data within the specified date range
start_date = "2010-05-30"
end_date = "2018-02-09"
df = df[(df["Date"] >= start_date) & (df["Date"] <= end_date)]

# Print final number of rows
final_count = len(df)
print(f"Final number of rows after filtering: {final_count}")

# Assign back to 'data'
data = df

# Drop 'Date' column
data.drop(["Date"], axis=1, inplace=True)

# Print final dataset
print(data.head())


In [ ]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, explained_variance_score
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, Bidirectional, LSTM, Lambda, Concatenate, Add, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# ---------------------------
# 1. Set Seed for Reproducibility
# ---------------------------
def set_random_seed(seed=20):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.config.experimental.enable_op_determinism()

set_random_seed(20)

# ---------------------------
# 2. Load & Preprocess Data
# ---------------------------
# For demonstration, we use a basic univariate "Close" series.
# Ensure that your DataFrame 'data' contains at least a "Close" column.
df = data[["Close"]].copy()
scaler = MinMaxScaler()
df["Close"] = scaler.fit_transform(df)

# ---------------------------
# 3. Create 15-day Sequences
# ---------------------------
def create_sequences(data, time_steps=15):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i : i + time_steps])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

X, y = create_sequences(df["Close"].values, time_steps=15)

# Split data: 70% train, 10% validation, 20% test.
train_size = int(len(X) * 0.7)
val_size = int(len(X) * 0.1)
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val     = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test   = X[train_size+val_size:], y[train_size+val_size:]

# Reshape each sample to (time_steps, 1)
X_train = X_train.reshape(-1, 15, 1)
X_val   = X_val.reshape(-1, 15, 1)
X_test  = X_test.reshape(-1, 15, 1)

# ---------------------------
# 4. Build Multi-Scale Hybrid Model
# ---------------------------
from tensorflow.keras.layers import Multiply, Activation

def build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, 1))

    # *Branch 1: Full 15-Day Analysis (Long-Term Trends)*
    full_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(inputs)
    full_branch = Dense(64, activation='relu')(full_branch)
    long_term_return = Dense(1, activation='tanh')(full_branch)
    long_term_return = Lambda(lambda x: x * scale_factor)(long_term_return)

    # *Branch 2: Recent 7-Day Analysis (Mid-Term Trends)*
    recent_input = Lambda(lambda x: x[:, -7:, :])(inputs)  # Take the last 7 days
    recent_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(recent_input)
    recent_branch = Dense(64, activation='relu')(recent_branch)
    mid_term_return = Dense(1, activation='tanh')(recent_branch)
    mid_term_return = Lambda(lambda x: x * scale_factor)(mid_term_return)

    # *Branch 3: Very Recent 1-Day Analysis (Short-Term Trends)*
    very_recent_input = Lambda(lambda x: x[:, -1:, :])(inputs)  # Take the last day's price
    very_recent_branch = Flatten()(very_recent_input)
    very_recent_branch = Dense(32, activation='relu')(very_recent_branch)
    short_term_return = Dense(1, activation='tanh')(very_recent_branch)
    short_term_return = Lambda(lambda x: x * recent_scale)(short_term_return)

    # *Gating Mechanism (Forget Gate)*
    gate_7 = Dense(1, activation='sigmoid')(mid_term_return)  # Gate for 7-day impact
    gate_1 = Dense(1, activation='sigmoid')(short_term_return)  # Gate for 1-day impact

    gated_mid_term_return = Multiply()([mid_term_return, gate_7])  # 7-day result scaled by gate
    gated_short_term_return = Multiply()([short_term_return, gate_1])  # 1-day result scaled by gate

    # *Combine Long-Term, Gated Mid-Term, and Gated Short-Term Predictions*
    fused_returns = Add()([long_term_return, gated_mid_term_return, gated_short_term_return])
    fused_returns = Dense(32, activation='relu')(fused_returns)
    final_return = Dense(1, activation='tanh')(fused_returns)
    final_return = Lambda(lambda x: x * scale_factor)(final_return)

    # *Use the last day's price for the final output calculation*
    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, final_return])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

r2 = r2_score(y_test_inv, y_pred_inv)
rmse = mean_squared_error(y_test_inv, y_pred_inv, squared=False)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mape = mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100
evs = explained_variance_score(y_test_inv, y_pred_inv)

print(f"📊 Final Results - R²: {r2:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}, MAPE: {mape:.2f}%, EVS: {evs:.4f}")

plt.figure(figsize=(10,5))
plt.plot(y_test_inv, label='Actual')
plt.plot(y_pred_inv, label='Predicted')
plt.xlabel("Time")
plt.ylabel("Close Price")
plt.legend()
plt.show()

In [ ]:
r2 = r2_score(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mse = mean_squared_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Calculate the actual range of the test set
actual_range = np.max(y_test_inv) - np.min(y_test_inv)

# Compute Scaled Metrics
mae_scaled = mae / actual_range
mse_scaled = mse / (actual_range ** 2)
rmse_scaled = rmse / actual_range

# Explained Variance Score
evs = explained_variance_score(y_test_inv, y_pred_inv) 

# Print the results
print(f"📊 Final Results -")
print(f"  R²: {r2:.4f}")
print(f"  MAE: {mae:.4f}, MAE Scaled: {mae_scaled:.4f}")
print(f"  MSE: {mse:.4f}, MSE Scaled: {mse_scaled:.4f}")
print(f"  RMSE: {rmse:.4f}, RMSE Scaled: {rmse_scaled:.4f}")
print(f"  MAPE: {mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100:.2f}%")
print(f"  EVS: {evs:.4f}")

## Working with Paper 3
* This paper does prediction on Nifty50 and Sensex
* The data is collected daily from 2013 to 2022
* The metric used is RMSE
* Their result is NIFTY50:-RMSE - 170.843,SENSEX:-RMSE - 578.746
* Paper link:-https://www.sciencedirect.com/science/article/pii/S1568494624005337

In [ ]:
vix = pd.read_csv("/kaggle/input/personal-dateset-vix/vix paper1.csv")
vix.drop("Vol.",axis=1,inplace=True)
vix["Change %"] = vix["Change %"].str.replace("%", "").astype(float)
vix.head(),vix.dtypes

In [ ]:
data=pd.read_csv("/kaggle/input/personal-dataset/Sensex Paper1.csv")
data=data[::-1]
data.reset_index(drop=True, inplace=True)
data.head()
data.nunique()

data.sort_index(axis=1,ascending=True)
data.rename(columns={'Price': 'Close'}, inplace=True)
df = data.copy()  # Ensure we don't modify the original dataset

# Convert financial columns to numeric (remove commas)
for col in ["Close", "Open", "High", "Low"]:
    df[col] = df[col].astype(str).str.replace(",", "").astype(float)

# Function to convert 'Vol.' column
def convert_volume(vol):
    if isinstance(vol, str):  # Ensure it's a string before replacing
        vol = vol.replace(",", "")  # Remove any thousand separators
        if "B" in vol:
            return float(vol.replace("B", "")) * 1_000_000_000
        elif "M" in vol:
            return float(vol.replace("M", "")) * 1_000_000
        elif "K" in vol:
            return float(vol.replace("K", "")) * 1_000
    return float(vol)  # Convert directly if already a number

df["Vol."] = df["Vol."].astype(str).apply(convert_volume)

# Convert 'Change %' column (remove '%' and convert to float)
df["Change %"] = df["Change %"].astype(str).str.replace("%", "").astype(float)

# Print final DataFrame
print(df.dtypes)
print(df)

# Assign back to 'data' (if needed)
data = df

data = pd.merge(data, vix, on='Date', how='inner')
data.drop(['Date'], axis=1, inplace=True)
data.head()

In [ ]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, explained_variance_score
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, Bidirectional, LSTM, Lambda, Concatenate, Add, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# ---------------------------
# 1. Set Seed for Reproducibility
# ---------------------------
def set_random_seed(seed=20):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.config.experimental.enable_op_determinism()

set_random_seed(20)

# ---------------------------
# 2. Load & Preprocess Data
# ---------------------------
# For demonstration, we use a basic univariate "Close" series.
# Ensure that your DataFrame 'data' contains at least a "Close" column.
df = data[["Close"]].copy()
scaler = MinMaxScaler()
df["Close"] = scaler.fit_transform(df)

# ---------------------------
# 3. Create 15-day Sequences
# ---------------------------
def create_sequences(data, time_steps=15):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i : i + time_steps])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

X, y = create_sequences(df["Close"].values, time_steps=15)

# Split data: 70% train, 10% validation, 20% test.
train_size = int(len(X) * 0.7)
val_size = int(len(X) * 0.1)
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val     = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test   = X[train_size+val_size:], y[train_size+val_size:]

# Reshape each sample to (time_steps, 1)
X_train = X_train.reshape(-1, 15, 1)
X_val   = X_val.reshape(-1, 15, 1)
X_test  = X_test.reshape(-1, 15, 1)

# ---------------------------
# 4. Build Multi-Scale Hybrid Model
# ---------------------------
from tensorflow.keras.layers import Multiply, Activation

def build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, 1))

    # *Branch 1: Full 15-Day Analysis (Long-Term Trends)*
    full_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(inputs)
    full_branch = Dense(64, activation='relu')(full_branch)
    long_term_return = Dense(1, activation='tanh')(full_branch)
    long_term_return = Lambda(lambda x: x * scale_factor)(long_term_return)

    # *Branch 2: Recent 7-Day Analysis (Mid-Term Trends)*
    recent_input = Lambda(lambda x: x[:, -7:, :])(inputs)  # Take the last 7 days
    recent_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(recent_input)
    recent_branch = Dense(64, activation='relu')(recent_branch)
    mid_term_return = Dense(1, activation='tanh')(recent_branch)
    mid_term_return = Lambda(lambda x: x * scale_factor)(mid_term_return)

    # *Branch 3: Very Recent 1-Day Analysis (Short-Term Trends)*
    very_recent_input = Lambda(lambda x: x[:, -1:, :])(inputs)  # Take the last day's price
    very_recent_branch = Flatten()(very_recent_input)
    very_recent_branch = Dense(32, activation='relu')(very_recent_branch)
    short_term_return = Dense(1, activation='tanh')(very_recent_branch)
    short_term_return = Lambda(lambda x: x * recent_scale)(short_term_return)

    # *Gating Mechanism (Forget Gate)*
    gate_7 = Dense(1, activation='sigmoid')(mid_term_return)  # Gate for 7-day impact
    gate_1 = Dense(1, activation='sigmoid')(short_term_return)  # Gate for 1-day impact

    gated_mid_term_return = Multiply()([mid_term_return, gate_7])  # 7-day result scaled by gate
    gated_short_term_return = Multiply()([short_term_return, gate_1])  # 1-day result scaled by gate

    # *Combine Long-Term, Gated Mid-Term, and Gated Short-Term Predictions*
    fused_returns = Add()([long_term_return, gated_mid_term_return, gated_short_term_return])
    fused_returns = Dense(32, activation='relu')(fused_returns)
    final_return = Dense(1, activation='tanh')(fused_returns)
    final_return = Lambda(lambda x: x * scale_factor)(final_return)

    # *Use the last day's price for the final output calculation*
    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, final_return])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

r2 = r2_score(y_test_inv, y_pred_inv)
rmse = mean_squared_error(y_test_inv, y_pred_inv, squared=False)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mape = mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100
evs = explained_variance_score(y_test_inv, y_pred_inv)

print(f"📊 Final Results - R²: {r2:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}, MAPE: {mape:.2f}%, EVS: {evs:.4f}")

plt.figure(figsize=(10,5))
plt.plot(y_test_inv, label='Actual')
plt.plot(y_pred_inv, label='Predicted')
plt.xlabel("Time")
plt.ylabel("Close Price")
plt.legend()
plt.show()

## doing without vix

In [ ]:
data=pd.read_csv("/kaggle/input/personal-dataset/Sensex Paper1.csv")
data=data[::-1]
data.reset_index(drop=True, inplace=True)
data.head()
data.nunique()

data.sort_index(axis=1,ascending=True)
data.rename(columns={'Price': 'Close'}, inplace=True)
df = data.copy()  # Ensure we don't modify the original dataset

# Convert financial columns to numeric (remove commas)
for col in ["Close", "Open", "High", "Low"]:
    df[col] = df[col].astype(str).str.replace(",", "").astype(float)

# Function to convert 'Vol.' column
def convert_volume(vol):
    if isinstance(vol, str):  # Ensure it's a string before replacing
        vol = vol.replace(",", "")  # Remove any thousand separators
        if "B" in vol:
            return float(vol.replace("B", "")) * 1_000_000_000
        elif "M" in vol:
            return float(vol.replace("M", "")) * 1_000_000
        elif "K" in vol:
            return float(vol.replace("K", "")) * 1_000
    return float(vol)  # Convert directly if already a number

df["Vol."] = df["Vol."].astype(str).apply(convert_volume)

# Convert 'Change %' column (remove '%' and convert to float)
df["Change %"] = df["Change %"].astype(str).str.replace("%", "").astype(float)

# Print final DataFrame
print(df.dtypes)
print(df)

# Assign back to 'data' (if needed)
data = df

#data = pd.merge(data, vix, on='Date', how='inner')
data.drop(['Date'], axis=1, inplace=True)
data.head()

In [ ]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, explained_variance_score
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, Bidirectional, LSTM, Lambda, Concatenate, Add, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# ---------------------------
# 1. Set Seed for Reproducibility
# ---------------------------
def set_random_seed(seed=20):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.config.experimental.enable_op_determinism()

set_random_seed(20)

# ---------------------------
# 2. Load & Preprocess Data
# ---------------------------
# For demonstration, we use a basic univariate "Close" series.
# Ensure that your DataFrame 'data' contains at least a "Close" column.
df = data[["Close"]].copy()
scaler = MinMaxScaler()
df["Close"] = scaler.fit_transform(df)

# ---------------------------
# 3. Create 15-day Sequences
# ---------------------------
def create_sequences(data, time_steps=15):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i : i + time_steps])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

X, y = create_sequences(df["Close"].values, time_steps=15)

# Split data: 70% train, 10% validation, 20% test.
train_size = int(len(X) * 0.7)
val_size = int(len(X) * 0.1)
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val     = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test   = X[train_size+val_size:], y[train_size+val_size:]

# Reshape each sample to (time_steps, 1)
X_train = X_train.reshape(-1, 15, 1)
X_val   = X_val.reshape(-1, 15, 1)
X_test  = X_test.reshape(-1, 15, 1)

# ---------------------------
# 4. Build Multi-Scale Hybrid Model
# ---------------------------
from tensorflow.keras.layers import Multiply, Activation

def build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, 1))

    # *Branch 1: Full 15-Day Analysis (Long-Term Trends)*
    full_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(inputs)
    full_branch = Dense(64, activation='relu')(full_branch)
    long_term_return = Dense(1, activation='tanh')(full_branch)
    long_term_return = Lambda(lambda x: x * scale_factor)(long_term_return)

    # *Branch 2: Recent 7-Day Analysis (Mid-Term Trends)*
    recent_input = Lambda(lambda x: x[:, -7:, :])(inputs)  # Take the last 7 days
    recent_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(recent_input)
    recent_branch = Dense(64, activation='relu')(recent_branch)
    mid_term_return = Dense(1, activation='tanh')(recent_branch)
    mid_term_return = Lambda(lambda x: x * scale_factor)(mid_term_return)

    # *Branch 3: Very Recent 1-Day Analysis (Short-Term Trends)*
    very_recent_input = Lambda(lambda x: x[:, -1:, :])(inputs)  # Take the last day's price
    very_recent_branch = Flatten()(very_recent_input)
    very_recent_branch = Dense(32, activation='relu')(very_recent_branch)
    short_term_return = Dense(1, activation='tanh')(very_recent_branch)
    short_term_return = Lambda(lambda x: x * recent_scale)(short_term_return)

    # *Gating Mechanism (Forget Gate)*
    gate_7 = Dense(1, activation='sigmoid')(mid_term_return)  # Gate for 7-day impact
    gate_1 = Dense(1, activation='sigmoid')(short_term_return)  # Gate for 1-day impact

    gated_mid_term_return = Multiply()([mid_term_return, gate_7])  # 7-day result scaled by gate
    gated_short_term_return = Multiply()([short_term_return, gate_1])  # 1-day result scaled by gate

    # *Combine Long-Term, Gated Mid-Term, and Gated Short-Term Predictions*
    fused_returns = Add()([long_term_return, gated_mid_term_return, gated_short_term_return])
    fused_returns = Dense(32, activation='relu')(fused_returns)
    final_return = Dense(1, activation='tanh')(fused_returns)
    final_return = Lambda(lambda x: x * scale_factor)(final_return)

    # *Use the last day's price for the final output calculation*
    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, final_return])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

r2 = r2_score(y_test_inv, y_pred_inv)
rmse = mean_squared_error(y_test_inv, y_pred_inv, squared=False)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mape = mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100
evs = explained_variance_score(y_test_inv, y_pred_inv)

print(f"📊 Final Results - R²: {r2:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}, MAPE: {mape:.2f}%, EVS: {evs:.4f}")

plt.figure(figsize=(10,5))
plt.plot(y_test_inv, label='Actual')
plt.plot(y_pred_inv, label='Predicted')
plt.xlabel("Time")
plt.ylabel("Close Price")
plt.legend()
plt.show()

## Working with Nifty50

In [ ]:
data=pd.read_csv("/kaggle/input/personal-dataset/Nifty50 Paper1.csv")
data=data[::-1]
data.reset_index(drop=True, inplace=True)
data.head()
data.nunique()

data.sort_index(axis=1,ascending=True)
data.rename(columns={'Price': 'Close'}, inplace=True)
df = data.copy()  # Ensure we don't modify the original dataset
vix = pd.read_csv("/kaggle/input/personal-dateset-vix/vix paper1.csv")
vix.drop("Vol.",axis=1,inplace=True)
vix["Change %"] = vix["Change %"].str.replace("%", "").astype(float)
vix.head(),vix.dtypes
# Convert financial columns to numeric (remove commas)
for col in ["Close", "Open", "High", "Low"]:
    df[col] = df[col].astype(str).str.replace(",", "").astype(float)

# Function to convert 'Vol.' column
def convert_volume(vol):
    if isinstance(vol, str):  # Ensure it's a string before replacing
        vol = vol.replace(",", "")  # Remove any thousand separators
        if "B" in vol:
            return float(vol.replace("B", "")) * 1_000_000_000
        elif "M" in vol:
            return float(vol.replace("M", "")) * 1_000_000
        elif "K" in vol:
            return float(vol.replace("K", "")) * 1_000
    return float(vol)  # Convert directly if already a number

df["Vol."] = df["Vol."].astype(str).apply(convert_volume)

# Convert 'Change %' column (remove '%' and convert to float)
df["Change %"] = df["Change %"].astype(str).str.replace("%", "").astype(float)

# Print final DataFrame
print(df.dtypes)
print(df)

# Assign back to 'data' (if needed)
data = df

data = pd.merge(data, vix, on='Date', how='inner')
data.drop(['Date'], axis=1, inplace=True)
data.head()

In [ ]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, explained_variance_score
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, Bidirectional, LSTM, Lambda, Concatenate, Add, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# ---------------------------
# 1. Set Seed for Reproducibility
# ---------------------------
def set_random_seed(seed=20):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.config.experimental.enable_op_determinism()

set_random_seed(20)

# ---------------------------
# 2. Load & Preprocess Data
# ---------------------------
# For demonstration, we use a basic univariate "Close" series.
# Ensure that your DataFrame 'data' contains at least a "Close" column.
df = data[["Close"]].copy()
scaler = MinMaxScaler()
df["Close"] = scaler.fit_transform(df)

# ---------------------------
# 3. Create 15-day Sequences
# ---------------------------
def create_sequences(data, time_steps=15):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i : i + time_steps])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

X, y = create_sequences(df["Close"].values, time_steps=15)

# Split data: 70% train, 10% validation, 20% test.
train_size = int(len(X) * 0.7)
val_size = int(len(X) * 0.1)
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val     = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test   = X[train_size+val_size:], y[train_size+val_size:]

# Reshape each sample to (time_steps, 1)
X_train = X_train.reshape(-1, 15, 1)
X_val   = X_val.reshape(-1, 15, 1)
X_test  = X_test.reshape(-1, 15, 1)

# ---------------------------
# 4. Build Multi-Scale Hybrid Model
# ---------------------------
from tensorflow.keras.layers import Multiply, Activation

def build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, 1))

    # *Branch 1: Full 15-Day Analysis (Long-Term Trends)*
    full_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(inputs)
    full_branch = Dense(64, activation='relu')(full_branch)
    long_term_return = Dense(1, activation='tanh')(full_branch)
    long_term_return = Lambda(lambda x: x * scale_factor)(long_term_return)

    # *Branch 2: Recent 7-Day Analysis (Mid-Term Trends)*
    recent_input = Lambda(lambda x: x[:, -7:, :])(inputs)  # Take the last 7 days
    recent_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(recent_input)
    recent_branch = Dense(64, activation='relu')(recent_branch)
    mid_term_return = Dense(1, activation='tanh')(recent_branch)
    mid_term_return = Lambda(lambda x: x * scale_factor)(mid_term_return)

    # *Branch 3: Very Recent 1-Day Analysis (Short-Term Trends)*
    very_recent_input = Lambda(lambda x: x[:, -1:, :])(inputs)  # Take the last day's price
    very_recent_branch = Flatten()(very_recent_input)
    very_recent_branch = Dense(32, activation='relu')(very_recent_branch)
    short_term_return = Dense(1, activation='tanh')(very_recent_branch)
    short_term_return = Lambda(lambda x: x * recent_scale)(short_term_return)

    # *Gating Mechanism (Forget Gate)*
    gate_7 = Dense(1, activation='sigmoid')(mid_term_return)  # Gate for 7-day impact
    gate_1 = Dense(1, activation='sigmoid')(short_term_return)  # Gate for 1-day impact

    gated_mid_term_return = Multiply()([mid_term_return, gate_7])  # 7-day result scaled by gate
    gated_short_term_return = Multiply()([short_term_return, gate_1])  # 1-day result scaled by gate

    # *Combine Long-Term, Gated Mid-Term, and Gated Short-Term Predictions*
    fused_returns = Add()([long_term_return, gated_mid_term_return, gated_short_term_return])
    fused_returns = Dense(32, activation='relu')(fused_returns)
    final_return = Dense(1, activation='tanh')(fused_returns)
    final_return = Lambda(lambda x: x * scale_factor)(final_return)

    # *Use the last day's price for the final output calculation*
    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, final_return])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

r2 = r2_score(y_test_inv, y_pred_inv)
rmse = mean_squared_error(y_test_inv, y_pred_inv, squared=False)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mape = mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100
evs = explained_variance_score(y_test_inv, y_pred_inv)

print(f"📊 Final Results - R²: {r2:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}, MAPE: {mape:.2f}%, EVS: {evs:.4f}")

plt.figure(figsize=(10,5))
plt.plot(y_test_inv, label='Actual')
plt.plot(y_pred_inv, label='Predicted')
plt.xlabel("Time")
plt.ylabel("Close Price")
plt.legend()
plt.show()

## doing without vix

In [ ]:
data=pd.read_csv("/kaggle/input/personal-dataset/Nifty50 Paper1.csv")
data=data[::-1]
data.reset_index(drop=True, inplace=True)
data.head()
data.nunique()

data.sort_index(axis=1,ascending=True)
data.rename(columns={'Price': 'Close'}, inplace=True)
df = data.copy()  # Ensure we don't modify the original dataset

# Convert financial columns to numeric (remove commas)
for col in ["Close", "Open", "High", "Low"]:
    df[col] = df[col].astype(str).str.replace(",", "").astype(float)

# Function to convert 'Vol.' column
def convert_volume(vol):
    if isinstance(vol, str):  # Ensure it's a string before replacing
        vol = vol.replace(",", "")  # Remove any thousand separators
        if "B" in vol:
            return float(vol.replace("B", "")) * 1_000_000_000
        elif "M" in vol:
            return float(vol.replace("M", "")) * 1_000_000
        elif "K" in vol:
            return float(vol.replace("K", "")) * 1_000
    return float(vol)  # Convert directly if already a number

df["Vol."] = df["Vol."].astype(str).apply(convert_volume)

# Convert 'Change %' column (remove '%' and convert to float)
df["Change %"] = df["Change %"].astype(str).str.replace("%", "").astype(float)

# Print final DataFrame
print(df.dtypes)
print(df)

# Assign back to 'data' (if needed)
data = df

#data = pd.merge(data, vix, on='Date', how='inner')
data.drop(['Date'], axis=1, inplace=True)
data.head()

In [ ]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, explained_variance_score
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, Bidirectional, LSTM, Lambda, Concatenate, Add, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# ---------------------------
# 1. Set Seed for Reproducibility
# ---------------------------
def set_random_seed(seed=20):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.config.experimental.enable_op_determinism()

set_random_seed(20)

# ---------------------------
# 2. Load & Preprocess Data
# ---------------------------
# For demonstration, we use a basic univariate "Close" series.
# Ensure that your DataFrame 'data' contains at least a "Close" column.
df = data[["Close"]].copy()
scaler = MinMaxScaler()
df["Close"] = scaler.fit_transform(df)

# ---------------------------
# 3. Create 15-day Sequences
# ---------------------------
def create_sequences(data, time_steps=15):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i : i + time_steps])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

X, y = create_sequences(df["Close"].values, time_steps=15)

# Split data: 70% train, 10% validation, 20% test.
train_size = int(len(X) * 0.7)
val_size = int(len(X) * 0.1)
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val     = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test   = X[train_size+val_size:], y[train_size+val_size:]

# Reshape each sample to (time_steps, 1)
X_train = X_train.reshape(-1, 15, 1)
X_val   = X_val.reshape(-1, 15, 1)
X_test  = X_test.reshape(-1, 15, 1)

# ---------------------------
# 4. Build Multi-Scale Hybrid Model
# ---------------------------
from tensorflow.keras.layers import Multiply, Activation

def build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, 1))

    # *Branch 1: Full 15-Day Analysis (Long-Term Trends)*
    full_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(inputs)
    full_branch = Dense(64, activation='relu')(full_branch)
    long_term_return = Dense(1, activation='tanh')(full_branch)
    long_term_return = Lambda(lambda x: x * scale_factor)(long_term_return)

    # *Branch 2: Recent 7-Day Analysis (Mid-Term Trends)*
    recent_input = Lambda(lambda x: x[:, -7:, :])(inputs)  # Take the last 7 days
    recent_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(recent_input)
    recent_branch = Dense(64, activation='relu')(recent_branch)
    mid_term_return = Dense(1, activation='tanh')(recent_branch)
    mid_term_return = Lambda(lambda x: x * scale_factor)(mid_term_return)

    # *Branch 3: Very Recent 1-Day Analysis (Short-Term Trends)*
    very_recent_input = Lambda(lambda x: x[:, -1:, :])(inputs)  # Take the last day's price
    very_recent_branch = Flatten()(very_recent_input)
    very_recent_branch = Dense(32, activation='relu')(very_recent_branch)
    short_term_return = Dense(1, activation='tanh')(very_recent_branch)
    short_term_return = Lambda(lambda x: x * recent_scale)(short_term_return)

    # *Gating Mechanism (Forget Gate)*
    gate_7 = Dense(1, activation='sigmoid')(mid_term_return)  # Gate for 7-day impact
    gate_1 = Dense(1, activation='sigmoid')(short_term_return)  # Gate for 1-day impact

    gated_mid_term_return = Multiply()([mid_term_return, gate_7])  # 7-day result scaled by gate
    gated_short_term_return = Multiply()([short_term_return, gate_1])  # 1-day result scaled by gate

    # *Combine Long-Term, Gated Mid-Term, and Gated Short-Term Predictions*
    fused_returns = Add()([long_term_return, gated_mid_term_return, gated_short_term_return])
    fused_returns = Dense(32, activation='relu')(fused_returns)
    final_return = Dense(1, activation='tanh')(fused_returns)
    final_return = Lambda(lambda x: x * scale_factor)(final_return)

    # *Use the last day's price for the final output calculation*
    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, final_return])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

r2 = r2_score(y_test_inv, y_pred_inv)
rmse = mean_squared_error(y_test_inv, y_pred_inv, squared=False)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mape = mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100
evs = explained_variance_score(y_test_inv, y_pred_inv)

print(f"📊 Final Results - R²: {r2:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}, MAPE: {mape:.2f}%, EVS: {evs:.4f}")

plt.figure(figsize=(10,5))
plt.plot(y_test_inv, label='Actual')
plt.plot(y_pred_inv, label='Predicted')
plt.xlabel("Time")
plt.ylabel("Close Price")
plt.legend()
plt.show()

## Working with Paper 4
* This paper does prediction on Nifty50
* The metric used is RMSE
* Their result is RMSE - 171.4
* We are training the model on data from 1st January 2020 till 31st December 2023 and testing it on data from 1st January
  2024 till 15th March 2024.
* Paper link:-https://www.semanticscholar.org/paper/Closing-Price-Prediction-for-the-NIFTY-50-Index%3A-A-Singh-Shah/d0eca144e2cd8e86c57eca34e3d9f4943f3c45f2

In [ ]:
import pandas as pd
data=pd.read_csv("/kaggle/input/personal-dataset/Nifty 50 Train paper 2.csv")
data2=pd.read_csv("/kaggle/input/personal-dataset/Nifty 50 Test paper2.csv")

In [ ]:
data.head(),data2.head()

In [ ]:
def preprocess(data):
    data=data[::-1]
    data.reset_index(drop=True, inplace=True)
    print(data.head())
    data.nunique()

    data.sort_index(axis=1,ascending=True)
    df=data
    print(df)
    data.rename(columns={'Price': 'Close'}, inplace=True)
    print(data.head()),print(data.dtypes)
    df = data.copy()  # Ensure we don't modify the original dataset

# Convert financial columns to numeric (remove commas)
    for col in ["Close", "Open", "High", "Low"]:
        df[col] = df[col].astype(str).str.replace(",", "").astype(float)

# Function to convert 'Vol.' column
    def convert_volume(vol):
        if isinstance(vol, str):  # Ensure it's a string before replacing
            vol = vol.replace(",", "")  # Remove any thousand separators
            if "B" in vol:
                return float(vol.replace("B", "")) * 1_000_000_000
            elif "M" in vol:
                return float(vol.replace("M", "")) * 1_000_000
            elif "K" in vol:
                return float(vol.replace("K", "")) * 1_000
        return float(vol)  # Convert directly if already a number

    df["Vol."] = df["Vol."].astype(str).apply(convert_volume)

# Convert 'Change %' column (remove '%' and convert to float)
    df["Change %"] = df["Change %"].astype(str).str.replace("%", "").astype(float)

# Print final DataFrame
    print(df.dtypes)
    print(df)

# Assign back to 'data' (if needed)
    data = df
    return data
    #data=df
#data.drop("Date",axis=1,inplace=True)

data=preprocess(data)


In [ ]:
data2=preprocess(data2)

In [ ]:
vix = pd.read_csv("/kaggle/input/vix-paper2/vix paper 2.csv")
vix.drop("Vol.",axis=1,inplace=True)
vix["Change %"] = vix["Change %"].str.replace("%", "").astype(float)
vix.head(),vix.dtypes

In [ ]:
data = pd.merge(data, vix, on='Date', how='inner')
data.drop("Date",axis=1,inplace=True)
print(data.head())


In [ ]:
data2 = pd.merge(data2, vix, on='Date', how='inner')
data2.drop("Date",axis=1,inplace=True)
print(data2.head())
print(len(data2))

In [ ]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, explained_variance_score
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, Bidirectional, LSTM, Lambda, Concatenate, Add, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# ---------------------------
# 1. Set Seed for Reproducibility
# ---------------------------
def set_random_seed(seed=20):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.config.experimental.enable_op_determinism()

set_random_seed(20)

# ---------------------------
# 2. Load & Preprocess Data
# ---------------------------
# For demonstration, we use a basic univariate "Close" series.
# Ensure that your DataFrame 'data' contains at least a "Close" column.
df = data[["Close"]].copy()
scaler = MinMaxScaler()
df["Close"] = scaler.fit_transform(df)

# ---------------------------
# 3. Create 15-day Sequences
# ---------------------------
def create_sequences(data, time_steps=15):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i : i + time_steps])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

X, y = create_sequences(df["Close"].values, time_steps=15)

# Split data: 70% train, 10% validation, 20% test.
train_size = int(len(X) * 0.7)
val_size = int(len(X) * 0.1)
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val     = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test   = X[train_size+val_size:], y[train_size+val_size:]

# Reshape each sample to (time_steps, 1)
X_train = X_train.reshape(-1, 15, 1)
X_val   = X_val.reshape(-1, 15, 1)
X_test  = X_test.reshape(-1, 15, 1)

# ---------------------------
# 4. Build Multi-Scale Hybrid Model
# ---------------------------
from tensorflow.keras.layers import Multiply, Activation

def build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, 1))

    # *Branch 1: Full 15-Day Analysis (Long-Term Trends)*
    full_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(inputs)
    full_branch = Dense(64, activation='relu')(full_branch)
    long_term_return = Dense(1, activation='tanh')(full_branch)
    long_term_return = Lambda(lambda x: x * scale_factor)(long_term_return)

    # *Branch 2: Recent 7-Day Analysis (Mid-Term Trends)*
    recent_input = Lambda(lambda x: x[:, -7:, :])(inputs)  # Take the last 7 days
    recent_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(recent_input)
    recent_branch = Dense(64, activation='relu')(recent_branch)
    mid_term_return = Dense(1, activation='tanh')(recent_branch)
    mid_term_return = Lambda(lambda x: x * scale_factor)(mid_term_return)

    # *Branch 3: Very Recent 1-Day Analysis (Short-Term Trends)*
    very_recent_input = Lambda(lambda x: x[:, -1:, :])(inputs)  # Take the last day's price
    very_recent_branch = Flatten()(very_recent_input)
    very_recent_branch = Dense(32, activation='relu')(very_recent_branch)
    short_term_return = Dense(1, activation='tanh')(very_recent_branch)
    short_term_return = Lambda(lambda x: x * recent_scale)(short_term_return)

    # *Gating Mechanism (Forget Gate)*
    gate_7 = Dense(1, activation='sigmoid')(mid_term_return)  # Gate for 7-day impact
    gate_1 = Dense(1, activation='sigmoid')(short_term_return)  # Gate for 1-day impact

    gated_mid_term_return = Multiply()([mid_term_return, gate_7])  # 7-day result scaled by gate
    gated_short_term_return = Multiply()([short_term_return, gate_1])  # 1-day result scaled by gate

    # *Combine Long-Term, Gated Mid-Term, and Gated Short-Term Predictions*
    fused_returns = Add()([long_term_return, gated_mid_term_return, gated_short_term_return])
    fused_returns = Dense(32, activation='relu')(fused_returns)
    final_return = Dense(1, activation='tanh')(fused_returns)
    final_return = Lambda(lambda x: x * scale_factor)(final_return)

    # *Use the last day's price for the final output calculation*
    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, final_return])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

r2 = r2_score(y_test_inv, y_pred_inv)
rmse = mean_squared_error(y_test_inv, y_pred_inv, squared=False)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mape = mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100
evs = explained_variance_score(y_test_inv, y_pred_inv)

print(f"📊 Final Results - R²: {r2:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}, MAPE: {mape:.2f}%, EVS: {evs:.4f}")

plt.figure(figsize=(10,5))
plt.plot(y_test_inv, label='Actual')
plt.plot(y_pred_inv, label='Predicted')
plt.xlabel("Time")
plt.ylabel("Close Price")
plt.legend()
plt.show()

## doing without vix

In [ ]:
data=pd.read_csv("/kaggle/input/personal-dataset/Nifty 50 Train paper 2.csv")
data2=pd.read_csv("/kaggle/input/personal-dataset/Nifty 50 Test paper2.csv")
data=preprocess(data)
data2=preprocess(data2)
data2.drop("Date",axis=1,inplace=True)
data.drop("Date",axis=1,inplace=True)

In [ ]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, explained_variance_score
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, Bidirectional, LSTM, Lambda, Concatenate, Add, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# ---------------------------
# 1. Set Seed for Reproducibility
# ---------------------------
def set_random_seed(seed=20):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.config.experimental.enable_op_determinism()

set_random_seed(20)

# ---------------------------
# 2. Load & Preprocess Data
# ---------------------------
# For demonstration, we use a basic univariate "Close" series.
# Ensure that your DataFrame 'data' contains at least a "Close" column.
df = data[["Close"]].copy()
scaler = MinMaxScaler()
df["Close"] = scaler.fit_transform(df)

# ---------------------------
# 3. Create 15-day Sequences
# ---------------------------
def create_sequences(data, time_steps=15):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i : i + time_steps])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

X, y = create_sequences(df["Close"].values, time_steps=15)

# Split data: 70% train, 10% validation, 20% test.
train_size = int(len(X) * 0.7)
val_size = int(len(X) * 0.1)
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val     = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test   = X[train_size+val_size:], y[train_size+val_size:]

# Reshape each sample to (time_steps, 1)
X_train = X_train.reshape(-1, 15, 1)
X_val   = X_val.reshape(-1, 15, 1)
X_test  = X_test.reshape(-1, 15, 1)

# ---------------------------
# 4. Build Multi-Scale Hybrid Model
# ---------------------------
from tensorflow.keras.layers import Multiply, Activation

def build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, 1))

    # *Branch 1: Full 15-Day Analysis (Long-Term Trends)*
    full_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(inputs)
    full_branch = Dense(64, activation='relu')(full_branch)
    long_term_return = Dense(1, activation='tanh')(full_branch)
    long_term_return = Lambda(lambda x: x * scale_factor)(long_term_return)

    # *Branch 2: Recent 7-Day Analysis (Mid-Term Trends)*
    recent_input = Lambda(lambda x: x[:, -7:, :])(inputs)  # Take the last 7 days
    recent_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(recent_input)
    recent_branch = Dense(64, activation='relu')(recent_branch)
    mid_term_return = Dense(1, activation='tanh')(recent_branch)
    mid_term_return = Lambda(lambda x: x * scale_factor)(mid_term_return)

    # *Branch 3: Very Recent 1-Day Analysis (Short-Term Trends)*
    very_recent_input = Lambda(lambda x: x[:, -1:, :])(inputs)  # Take the last day's price
    very_recent_branch = Flatten()(very_recent_input)
    very_recent_branch = Dense(32, activation='relu')(very_recent_branch)
    short_term_return = Dense(1, activation='tanh')(very_recent_branch)
    short_term_return = Lambda(lambda x: x * recent_scale)(short_term_return)

    # *Gating Mechanism (Forget Gate)*
    gate_7 = Dense(1, activation='sigmoid')(mid_term_return)  # Gate for 7-day impact
    gate_1 = Dense(1, activation='sigmoid')(short_term_return)  # Gate for 1-day impact

    gated_mid_term_return = Multiply()([mid_term_return, gate_7])  # 7-day result scaled by gate
    gated_short_term_return = Multiply()([short_term_return, gate_1])  # 1-day result scaled by gate

    # *Combine Long-Term, Gated Mid-Term, and Gated Short-Term Predictions*
    fused_returns = Add()([long_term_return, gated_mid_term_return, gated_short_term_return])
    fused_returns = Dense(32, activation='relu')(fused_returns)
    final_return = Dense(1, activation='tanh')(fused_returns)
    final_return = Lambda(lambda x: x * scale_factor)(final_return)

    # *Use the last day's price for the final output calculation*
    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, final_return])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

r2 = r2_score(y_test_inv, y_pred_inv)
rmse = mean_squared_error(y_test_inv, y_pred_inv, squared=False)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mape = mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100
evs = explained_variance_score(y_test_inv, y_pred_inv)

print(f"📊 Final Results - R²: {r2:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}, MAPE: {mape:.2f}%, EVS: {evs:.4f}")

plt.figure(figsize=(10,5))
plt.plot(y_test_inv, label='Actual')
plt.plot(y_pred_inv, label='Predicted')
plt.xlabel("Time")
plt.ylabel("Close Price")
plt.legend()
plt.show()

## Working with Paper 5
* This paper does prediction on Nifty50 and BSE Sensex
* The ranges are:-
* ## BSE( Data )
 24/02/2015-27/11/2022

* ## NSE(Data)
 27/10/2006-27/11/2022
* The metric used are RMSE,MSE,MAE
* Their best results are:-
* ## BSE 15 Day:
MAE Scaled: 0.0522
 MSE Scaled: 0.0515
 RMSE Scaled: 0.1456
* ## NSE 15 Day:
MAE Scaled: 0.0712
 MSE Scaled: 0.0521
  RMSE Scaled: 0.1365
* Paper link:-https://ieeexplore.ieee.org/document/10481618

### Sensex (without vix)

In [ ]:

data=pd.read_csv("/kaggle/input/paper-5-deepstock-dataset/BSE Sensex 30 Historical Data ( paper - 5 ).csv")
data=data[::-1]
data.reset_index(drop=True, inplace=True)
data.head()
data.nunique()

data.sort_index(axis=1,ascending=True)
data.rename(columns={'Price': 'Close'}, inplace=True)
df = data.copy()  # Ensure we don't modify the original dataset

# Convert financial columns to numeric (remove commas)
for col in ["Close", "Open", "High", "Low"]:
    df[col] = df[col].astype(str).str.replace(",", "").astype(float)

# Function to convert 'Vol.' column
def convert_volume(vol):
    if isinstance(vol, str):  # Ensure it's a string before replacing
        vol = vol.replace(",", "")  # Remove any thousand separators
        if "B" in vol:
            return float(vol.replace("B", "")) * 1_000_000_000
        elif "M" in vol:
            return float(vol.replace("M", "")) * 1_000_000
        elif "K" in vol:
            return float(vol.replace("K", "")) * 1_000
    return float(vol)  # Convert directly if already a number

df["Vol."] = df["Vol."].astype(str).apply(convert_volume)

# Convert 'Change %' column (remove '%' and convert to float)
df["Change %"] = df["Change %"].astype(str).str.replace("%", "").astype(float)

# Print final DataFrame
print(df.dtypes)
print(df)

# Assign back to 'data' (if needed)
data = df

#data = pd.merge(data, vix, on='Date', how='inner')
data.drop(['Date'], axis=1, inplace=True)
data.head()

In [ ]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, explained_variance_score
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, Bidirectional, LSTM, Lambda, Concatenate, Add, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# ---------------------------
# 1. Set Seed for Reproducibility
# ---------------------------
def set_random_seed(seed=20):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.config.experimental.enable_op_determinism()

set_random_seed(20)

# ---------------------------
# 2. Load & Preprocess Data
# ---------------------------
# For demonstration, we use a basic univariate "Close" series.
# Ensure that your DataFrame 'data' contains at least a "Close" column.
df = data[["Close"]].copy()
scaler = MinMaxScaler()
df["Close"] = scaler.fit_transform(df)

# ---------------------------
# 3. Create 15-day Sequences
# ---------------------------
def create_sequences(data, time_steps=15):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i : i + time_steps])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

X, y = create_sequences(df["Close"].values, time_steps=15)

# Split data: 70% train, 10% validation, 20% test.
train_size = int(len(X) * 0.7)
val_size = int(len(X) * 0.1)
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val     = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test   = X[train_size+val_size:], y[train_size+val_size:]

# Reshape each sample to (time_steps, 1)
X_train = X_train.reshape(-1, 15, 1)
X_val   = X_val.reshape(-1, 15, 1)
X_test  = X_test.reshape(-1, 15, 1)

# ---------------------------
# 4. Build Multi-Scale Hybrid Model
# ---------------------------
from tensorflow.keras.layers import Multiply, Activation

def build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, 1))

    # *Branch 1: Full 15-Day Analysis (Long-Term Trends)*
    full_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(inputs)
    full_branch = Dense(64, activation='relu')(full_branch)
    long_term_return = Dense(1, activation='tanh')(full_branch)
    long_term_return = Lambda(lambda x: x * scale_factor)(long_term_return)

    # *Branch 2: Recent 7-Day Analysis (Mid-Term Trends)*
    recent_input = Lambda(lambda x: x[:, -7:, :])(inputs)  # Take the last 7 days
    recent_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(recent_input)
    recent_branch = Dense(64, activation='relu')(recent_branch)
    mid_term_return = Dense(1, activation='tanh')(recent_branch)
    mid_term_return = Lambda(lambda x: x * scale_factor)(mid_term_return)

    # *Branch 3: Very Recent 1-Day Analysis (Short-Term Trends)*
    very_recent_input = Lambda(lambda x: x[:, -1:, :])(inputs)  # Take the last day's price
    very_recent_branch = Flatten()(very_recent_input)
    very_recent_branch = Dense(32, activation='relu')(very_recent_branch)
    short_term_return = Dense(1, activation='tanh')(very_recent_branch)
    short_term_return = Lambda(lambda x: x * recent_scale)(short_term_return)

    # *Gating Mechanism (Forget Gate)*
    gate_7 = Dense(1, activation='sigmoid')(mid_term_return)  # Gate for 7-day impact
    gate_1 = Dense(1, activation='sigmoid')(short_term_return)  # Gate for 1-day impact

    gated_mid_term_return = Multiply()([mid_term_return, gate_7])  # 7-day result scaled by gate
    gated_short_term_return = Multiply()([short_term_return, gate_1])  # 1-day result scaled by gate

    # *Combine Long-Term, Gated Mid-Term, and Gated Short-Term Predictions*
    fused_returns = Add()([long_term_return, gated_mid_term_return, gated_short_term_return])
    fused_returns = Dense(32, activation='relu')(fused_returns)
    final_return = Dense(1, activation='tanh')(fused_returns)
    final_return = Lambda(lambda x: x * scale_factor)(final_return)

    # *Use the last day's price for the final output calculation*
    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, final_return])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
# Predictions on test set
##y_pred = model.predict(X_test)

# Inverse transform predictions and actual values
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

# Inverse transform predictions and actual values
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

# Calculate Metrics
r2 = r2_score(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mse = mean_squared_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Calculate the actual range of the test set
actual_range = np.max(y_test_inv) - np.min(y_test_inv)

# Compute Scaled Metrics
mae_scaled = mae / actual_range
mse_scaled = mse / (actual_range ** 2)
rmse_scaled = rmse / actual_range

# Explained Variance Score
evs = explained_variance_score(y_test_inv, y_pred_inv) 

# Print the results
print(f"📊 Final Results -")
print(f"  R²: {r2:.4f}")
print(f"  MAE: {mae:.4f}, MAE Scaled: {mae_scaled:.4f}")
print(f"  MSE: {mse:.4f}, MSE Scaled: {mse_scaled:.4f}")
print(f"  RMSE: {rmse:.4f}, RMSE Scaled: {rmse_scaled:.4f}")
print(f"  MAPE: {mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100:.2f}%")
print(f"  EVS: {evs:.4f}")

## Sensex with vix

In [ ]:
vix = pd.read_csv("/kaggle/input/paper-5-deepstock-dataset/India VIX Historical Data ( paper 5 ).csv")
vix.drop("Vol.",axis=1,inplace=True)
vix["Change %"] = vix["Change %"].str.replace("%", "").astype(float)
vix.head(),vix.dtypes
data=pd.read_csv("/kaggle/input/paper-5-deepstock-dataset/BSE Sensex 30 Historical Data ( paper - 5 ).csv")
data=data[::-1]
data.reset_index(drop=True, inplace=True)
data.head()
data.nunique()

data.sort_index(axis=1,ascending=True)
data.rename(columns={'Price': 'Close'}, inplace=True)
df = data.copy()  # Ensure we don't modify the original dataset

# Convert financial columns to numeric (remove commas)
for col in ["Close", "Open", "High", "Low"]:
    df[col] = df[col].astype(str).str.replace(",", "").astype(float)

# Function to convert 'Vol.' column
def convert_volume(vol):
    if isinstance(vol, str):  # Ensure it's a string before replacing
        vol = vol.replace(",", "")  # Remove any thousand separators
        if "B" in vol:
            return float(vol.replace("B", "")) * 1_000_000_000
        elif "M" in vol:
            return float(vol.replace("M", "")) * 1_000_000
        elif "K" in vol:
            return float(vol.replace("K", "")) * 1_000
    return float(vol)  # Convert directly if already a number

df["Vol."] = df["Vol."].astype(str).apply(convert_volume)

# Convert 'Change %' column (remove '%' and convert to float)
df["Change %"] = df["Change %"].astype(str).str.replace("%", "").astype(float)

# Print final DataFrame
print(df.dtypes)
print(df)

# Assign back to 'data' (if needed)
data = df

data = pd.merge(data, vix, on='Date', how='inner')
data.drop(['Date'], axis=1, inplace=True)
data.head()

In [ ]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, explained_variance_score
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, Bidirectional, LSTM, Lambda, Concatenate, Add, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# ---------------------------
# 1. Set Seed for Reproducibility
# ---------------------------
def set_random_seed(seed=20):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.config.experimental.enable_op_determinism()

set_random_seed(20)

# ---------------------------
# 2. Load & Preprocess Data
# ---------------------------
# For demonstration, we use a basic univariate "Close" series.
# Ensure that your DataFrame 'data' contains at least a "Close" column.
df = data[["Close"]].copy()
scaler = MinMaxScaler()
df["Close"] = scaler.fit_transform(df)

# ---------------------------
# 3. Create 15-day Sequences
# ---------------------------
def create_sequences(data, time_steps=15):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i : i + time_steps])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

X, y = create_sequences(df["Close"].values, time_steps=15)

# Split data: 70% train, 10% validation, 20% test.
train_size = int(len(X) * 0.7)
val_size = int(len(X) * 0.1)
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val     = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test   = X[train_size+val_size:], y[train_size+val_size:]

# Reshape each sample to (time_steps, 1)
X_train = X_train.reshape(-1, 15, 1)
X_val   = X_val.reshape(-1, 15, 1)
X_test  = X_test.reshape(-1, 15, 1)

# ---------------------------
# 4. Build Multi-Scale Hybrid Model
# ---------------------------
from tensorflow.keras.layers import Multiply, Activation

def build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, 1))

    # *Branch 1: Full 15-Day Analysis (Long-Term Trends)*
    full_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(inputs)
    full_branch = Dense(64, activation='relu')(full_branch)
    long_term_return = Dense(1, activation='tanh')(full_branch)
    long_term_return = Lambda(lambda x: x * scale_factor)(long_term_return)

    # *Branch 2: Recent 7-Day Analysis (Mid-Term Trends)*
    recent_input = Lambda(lambda x: x[:, -7:, :])(inputs)  # Take the last 7 days
    recent_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(recent_input)
    recent_branch = Dense(64, activation='relu')(recent_branch)
    mid_term_return = Dense(1, activation='tanh')(recent_branch)
    mid_term_return = Lambda(lambda x: x * scale_factor)(mid_term_return)

    # *Branch 3: Very Recent 1-Day Analysis (Short-Term Trends)*
    very_recent_input = Lambda(lambda x: x[:, -1:, :])(inputs)  # Take the last day's price
    very_recent_branch = Flatten()(very_recent_input)
    very_recent_branch = Dense(32, activation='relu')(very_recent_branch)
    short_term_return = Dense(1, activation='tanh')(very_recent_branch)
    short_term_return = Lambda(lambda x: x * recent_scale)(short_term_return)

    # *Gating Mechanism (Forget Gate)*
    gate_7 = Dense(1, activation='sigmoid')(mid_term_return)  # Gate for 7-day impact
    gate_1 = Dense(1, activation='sigmoid')(short_term_return)  # Gate for 1-day impact

    gated_mid_term_return = Multiply()([mid_term_return, gate_7])  # 7-day result scaled by gate
    gated_short_term_return = Multiply()([short_term_return, gate_1])  # 1-day result scaled by gate

    # *Combine Long-Term, Gated Mid-Term, and Gated Short-Term Predictions*
    fused_returns = Add()([long_term_return, gated_mid_term_return, gated_short_term_return])
    fused_returns = Dense(32, activation='relu')(fused_returns)
    final_return = Dense(1, activation='tanh')(fused_returns)
    final_return = Lambda(lambda x: x * scale_factor)(final_return)

    # *Use the last day's price for the final output calculation*
    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, final_return])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
# Predictions on test set
##y_pred = model.predict(X_test)

# Inverse transform predictions and actual values
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

# Inverse transform predictions and actual values
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

# Calculate Metrics
r2 = r2_score(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mse = mean_squared_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Calculate the actual range of the test set
actual_range = np.max(y_test_inv) - np.min(y_test_inv)

# Compute Scaled Metrics
mae_scaled = mae / actual_range
mse_scaled = mse / (actual_range ** 2)
rmse_scaled = rmse / actual_range

# Explained Variance Score
evs = explained_variance_score(y_test_inv, y_pred_inv) 

# Print the results
print(f"📊 Final Results -")
print(f"  R²: {r2:.4f}")
print(f"  MAE: {mae:.4f}, MAE Scaled: {mae_scaled:.4f}")
print(f"  MSE: {mse:.4f}, MSE Scaled: {mse_scaled:.4f}")
print(f"  RMSE: {rmse:.4f}, RMSE Scaled: {rmse_scaled:.4f}")
print(f"  MAPE: {mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100:.2f}%")
print(f"  EVS: {evs:.4f}")

## Nifty50 with vix

In [ ]:
vix = pd.read_csv("/kaggle/input/paper-5-deepstock-dataset/India VIX Historical Data ( paper 5 ).csv")
vix.drop("Vol.",axis=1,inplace=True)
vix["Change %"] = vix["Change %"].str.replace("%", "").astype(float)
vix.head(),vix.dtypes

In [ ]:
data=pd.read_csv("/kaggle/input/paper-5-deepstock-dataset/Nifty 50 Historical Data - paper( 5 ).csv")
data=data[::-1]
data.reset_index(drop=True, inplace=True)
data.head()
data.nunique()

data.sort_index(axis=1,ascending=True)
data.rename(columns={'Price': 'Close'}, inplace=True)
df = data.copy()  # Ensure we don't modify the original dataset

# Convert financial columns to numeric (remove commas)
for col in ["Close", "Open", "High", "Low"]:
    df[col] = df[col].astype(str).str.replace(",", "").astype(float)

# Function to convert 'Vol.' column
def convert_volume(vol):
    if isinstance(vol, str):  # Ensure it's a string before replacing
        vol = vol.replace(",", "")  # Remove any thousand separators
        if "B" in vol:
            return float(vol.replace("B", "")) * 1_000_000_000
        elif "M" in vol:
            return float(vol.replace("M", "")) * 1_000_000
        elif "K" in vol:
            return float(vol.replace("K", "")) * 1_000
    return float(vol)  # Convert directly if already a number

df["Vol."] = df["Vol."].astype(str).apply(convert_volume)

# Convert 'Change %' column (remove '%' and convert to float)
df["Change %"] = df["Change %"].astype(str).str.replace("%", "").astype(float)

# Print final DataFrame
print(df.dtypes)
print(df)

# Assign back to 'data' (if needed)
data = df

data = pd.merge(data, vix, on='Date', how='inner')
data.drop(['Date'], axis=1, inplace=True)
data.head()

In [ ]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, explained_variance_score
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, Bidirectional, LSTM, Lambda, Concatenate, Add, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# ---------------------------
# 1. Set Seed for Reproducibility
# ---------------------------
def set_random_seed(seed=20):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.config.experimental.enable_op_determinism()

set_random_seed(20)

# ---------------------------
# 2. Load & Preprocess Data
# ---------------------------
# For demonstration, we use a basic univariate "Close" series.
# Ensure that your DataFrame 'data' contains at least a "Close" column.
df = data[["Close"]].copy()
scaler = MinMaxScaler()
df["Close"] = scaler.fit_transform(df)

# ---------------------------
# 3. Create 15-day Sequences
# ---------------------------
def create_sequences(data, time_steps=15):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i : i + time_steps])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

X, y = create_sequences(df["Close"].values, time_steps=15)

# Split data: 70% train, 10% validation, 20% test.
train_size = int(len(X) * 0.7)
val_size = int(len(X) * 0.1)
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val     = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test   = X[train_size+val_size:], y[train_size+val_size:]

# Reshape each sample to (time_steps, 1)
X_train = X_train.reshape(-1, 15, 1)
X_val   = X_val.reshape(-1, 15, 1)
X_test  = X_test.reshape(-1, 15, 1)

# ---------------------------
# 4. Build Multi-Scale Hybrid Model
# ---------------------------
from tensorflow.keras.layers import Multiply, Activation

def build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, 1))

    # *Branch 1: Full 15-Day Analysis (Long-Term Trends)*
    full_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(inputs)
    full_branch = Dense(64, activation='relu')(full_branch)
    long_term_return = Dense(1, activation='tanh')(full_branch)
    long_term_return = Lambda(lambda x: x * scale_factor)(long_term_return)

    # *Branch 2: Recent 7-Day Analysis (Mid-Term Trends)*
    recent_input = Lambda(lambda x: x[:, -7:, :])(inputs)  # Take the last 7 days
    recent_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(recent_input)
    recent_branch = Dense(64, activation='relu')(recent_branch)
    mid_term_return = Dense(1, activation='tanh')(recent_branch)
    mid_term_return = Lambda(lambda x: x * scale_factor)(mid_term_return)

    # *Branch 3: Very Recent 1-Day Analysis (Short-Term Trends)*
    very_recent_input = Lambda(lambda x: x[:, -1:, :])(inputs)  # Take the last day's price
    very_recent_branch = Flatten()(very_recent_input)
    very_recent_branch = Dense(32, activation='relu')(very_recent_branch)
    short_term_return = Dense(1, activation='tanh')(very_recent_branch)
    short_term_return = Lambda(lambda x: x * recent_scale)(short_term_return)

    # *Gating Mechanism (Forget Gate)*
    gate_7 = Dense(1, activation='sigmoid')(mid_term_return)  # Gate for 7-day impact
    gate_1 = Dense(1, activation='sigmoid')(short_term_return)  # Gate for 1-day impact

    gated_mid_term_return = Multiply()([mid_term_return, gate_7])  # 7-day result scaled by gate
    gated_short_term_return = Multiply()([short_term_return, gate_1])  # 1-day result scaled by gate

    # *Combine Long-Term, Gated Mid-Term, and Gated Short-Term Predictions*
    fused_returns = Add()([long_term_return, gated_mid_term_return, gated_short_term_return])
    fused_returns = Dense(32, activation='relu')(fused_returns)
    final_return = Dense(1, activation='tanh')(fused_returns)
    final_return = Lambda(lambda x: x * scale_factor)(final_return)

    # *Use the last day's price for the final output calculation*
    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, final_return])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
# Predictions on test set
##y_pred = model.predict(X_test)

# Inverse transform predictions and actual values
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

# Inverse transform predictions and actual values
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

# Calculate Metrics
r2 = r2_score(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mse = mean_squared_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Calculate the actual range of the test set
actual_range = np.max(y_test_inv) - np.min(y_test_inv)

# Compute Scaled Metrics
mae_scaled = mae / actual_range
mse_scaled = mse / (actual_range ** 2)
rmse_scaled = rmse / actual_range

# Explained Variance Score
evs = explained_variance_score(y_test_inv, y_pred_inv) 

# Print the results
print(f"📊 Final Results -")
print(f"  R²: {r2:.4f}")
print(f"  MAE: {mae:.4f}, MAE Scaled: {mae_scaled:.4f}")
print(f"  MSE: {mse:.4f}, MSE Scaled: {mse_scaled:.4f}")
print(f"  RMSE: {rmse:.4f}, RMSE Scaled: {rmse_scaled:.4f}")
print(f"  MAPE: {mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100:.2f}%")
print(f"  EVS: {evs:.4f}")

## Nifty50 without vix

In [ ]:
data=pd.read_csv("/kaggle/input/paper-5-deepstock-dataset/Nifty 50 Historical Data - paper( 5 ).csv")
data=data[::-1]
data.reset_index(drop=True, inplace=True)
data.head()
data.nunique()

data.sort_index(axis=1,ascending=True)
data.rename(columns={'Price': 'Close'}, inplace=True)
df = data.copy()  # Ensure we don't modify the original dataset

# Convert financial columns to numeric (remove commas)
for col in ["Close", "Open", "High", "Low"]:
    df[col] = df[col].astype(str).str.replace(",", "").astype(float)

# Function to convert 'Vol.' column
def convert_volume(vol):
    if isinstance(vol, str):  # Ensure it's a string before replacing
        vol = vol.replace(",", "")  # Remove any thousand separators
        if "B" in vol:
            return float(vol.replace("B", "")) * 1_000_000_000
        elif "M" in vol:
            return float(vol.replace("M", "")) * 1_000_000
        elif "K" in vol:
            return float(vol.replace("K", "")) * 1_000
    return float(vol)  # Convert directly if already a number

df["Vol."] = df["Vol."].astype(str).apply(convert_volume)

# Convert 'Change %' column (remove '%' and convert to float)
df["Change %"] = df["Change %"].astype(str).str.replace("%", "").astype(float)

# Print final DataFrame
print(df.dtypes)
print(df)

# Assign back to 'data' (if needed)
data = df

#data = pd.merge(data, vix, on='Date', how='inner')
data.drop(['Date'], axis=1, inplace=True)
data.head()

In [ ]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, explained_variance_score
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, Bidirectional, LSTM, Lambda, Concatenate, Add, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# ---------------------------
# 1. Set Seed for Reproducibility
# ---------------------------
def set_random_seed(seed=20):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.config.experimental.enable_op_determinism()

set_random_seed(20)

# ---------------------------
# 2. Load & Preprocess Data
# ---------------------------
# For demonstration, we use a basic univariate "Close" series.
# Ensure that your DataFrame 'data' contains at least a "Close" column.
df = data[["Close"]].copy()
scaler = MinMaxScaler()
df["Close"] = scaler.fit_transform(df)

# ---------------------------
# 3. Create 15-day Sequences
# ---------------------------
def create_sequences(data, time_steps=15):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i : i + time_steps])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

X, y = create_sequences(df["Close"].values, time_steps=15)

# Split data: 70% train, 10% validation, 20% test.
train_size = int(len(X) * 0.7)
val_size = int(len(X) * 0.1)
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val     = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test   = X[train_size+val_size:], y[train_size+val_size:]

# Reshape each sample to (time_steps, 1)
X_train = X_train.reshape(-1, 15, 1)
X_val   = X_val.reshape(-1, 15, 1)
X_test  = X_test.reshape(-1, 15, 1)

# ---------------------------
# 4. Build Multi-Scale Hybrid Model
# ---------------------------
from tensorflow.keras.layers import Multiply, Activation

def build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, 1))

    # *Branch 1: Full 15-Day Analysis (Long-Term Trends)*
    full_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(inputs)
    full_branch = Dense(64, activation='relu')(full_branch)
    long_term_return = Dense(1, activation='tanh')(full_branch)
    long_term_return = Lambda(lambda x: x * scale_factor)(long_term_return)

    # *Branch 2: Recent 7-Day Analysis (Mid-Term Trends)*
    recent_input = Lambda(lambda x: x[:, -7:, :])(inputs)  # Take the last 7 days
    recent_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(recent_input)
    recent_branch = Dense(64, activation='relu')(recent_branch)
    mid_term_return = Dense(1, activation='tanh')(recent_branch)
    mid_term_return = Lambda(lambda x: x * scale_factor)(mid_term_return)

    # *Branch 3: Very Recent 1-Day Analysis (Short-Term Trends)*
    very_recent_input = Lambda(lambda x: x[:, -1:, :])(inputs)  # Take the last day's price
    very_recent_branch = Flatten()(very_recent_input)
    very_recent_branch = Dense(32, activation='relu')(very_recent_branch)
    short_term_return = Dense(1, activation='tanh')(very_recent_branch)
    short_term_return = Lambda(lambda x: x * recent_scale)(short_term_return)

    # *Gating Mechanism (Forget Gate)*
    gate_7 = Dense(1, activation='sigmoid')(mid_term_return)  # Gate for 7-day impact
    gate_1 = Dense(1, activation='sigmoid')(short_term_return)  # Gate for 1-day impact

    gated_mid_term_return = Multiply()([mid_term_return, gate_7])  # 7-day result scaled by gate
    gated_short_term_return = Multiply()([short_term_return, gate_1])  # 1-day result scaled by gate

    # *Combine Long-Term, Gated Mid-Term, and Gated Short-Term Predictions*
    fused_returns = Add()([long_term_return, gated_mid_term_return, gated_short_term_return])
    fused_returns = Dense(32, activation='relu')(fused_returns)
    final_return = Dense(1, activation='tanh')(fused_returns)
    final_return = Lambda(lambda x: x * scale_factor)(final_return)

    # *Use the last day's price for the final output calculation*
    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, final_return])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
# Predictions on test set
##y_pred = model.predict(X_test)

# Inverse transform predictions and actual values
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

# Inverse transform predictions and actual values
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

# Calculate Metrics
r2 = r2_score(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mse = mean_squared_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Calculate the actual range of the test set
actual_range = np.max(y_test_inv) - np.min(y_test_inv)

# Compute Scaled Metrics
mae_scaled = mae / actual_range
mse_scaled = mse / (actual_range ** 2)
rmse_scaled = rmse / actual_range

# Explained Variance Score
evs = explained_variance_score(y_test_inv, y_pred_inv) 

# Print the results
print(f"📊 Final Results -")
print(f"  R²: {r2:.4f}")
print(f"  MAE: {mae:.4f}, MAE Scaled: {mae_scaled:.4f}")
print(f"  MSE: {mse:.4f}, MSE Scaled: {mse_scaled:.4f}")
print(f"  RMSE: {rmse:.4f}, RMSE Scaled: {rmse_scaled:.4f}")
print(f"  MAPE: {mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100:.2f}%")
print(f"  EVS: {evs:.4f}")

# OUR RANGE: SENSEX (15-3-2008 to 15-3-2024)

In [ ]:
vix = pd.read_csv("/kaggle/input/our-data-15-3-2008-to-15-3-2024/India VIX Historical Data (3).csv")
vix.drop("Vol.",axis=1,inplace=True)
vix["Change %"] = vix["Change %"].str.replace("%", "").astype(float)
vix.head(),vix.dtypes
data=pd.read_csv("/kaggle/input/our-data-15-3-2008-to-15-3-2024/BSE Sensex 30 Historical Data (3).csv")
data=data[::-1]
data.reset_index(drop=True, inplace=True)
data.head()
data.nunique()

data.sort_index(axis=1,ascending=True)
data.rename(columns={'Price': 'Close'}, inplace=True)
df = data.copy()  # Ensure we don't modify the original dataset

# Convert financial columns to numeric (remove commas)
for col in ["Close", "Open", "High", "Low"]:
    df[col] = df[col].astype(str).str.replace(",", "").astype(float)

# Function to convert 'Vol.' column
def convert_volume(vol):
    if isinstance(vol, str):  # Ensure it's a string before replacing
        vol = vol.replace(",", "")  # Remove any thousand separators
        if "B" in vol:
            return float(vol.replace("B", "")) * 1_000_000_000
        elif "M" in vol:
            return float(vol.replace("M", "")) * 1_000_000
        elif "K" in vol:
            return float(vol.replace("K", "")) * 1_000
    return float(vol)  # Convert directly if already a number

df["Vol."] = df["Vol."].astype(str).apply(convert_volume)

# Convert 'Change %' column (remove '%' and convert to float)
df["Change %"] = df["Change %"].astype(str).str.replace("%", "").astype(float)

# Print final DataFrame
print(df.dtypes)
print(df)

# Assign back to 'data' (if needed)
data = df

data = pd.merge(data, vix, on='Date', how='inner')
data.drop(['Date'], axis=1, inplace=True)
data.head()

In [ ]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, explained_variance_score
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, Bidirectional, LSTM, Lambda, Concatenate, Add, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# ---------------------------
# 1. Set Seed for Reproducibility
# ---------------------------
def set_random_seed(seed=20):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.config.experimental.enable_op_determinism()

set_random_seed(20)

# ---------------------------
# 2. Load & Preprocess Data
# ---------------------------
# For demonstration, we use a basic univariate "Close" series.
# Ensure that your DataFrame 'data' contains at least a "Close" column.
df = data[["Close"]].copy()
scaler = MinMaxScaler()
df["Close"] = scaler.fit_transform(df)

# ---------------------------
# 3. Create 15-day Sequences
# ---------------------------
def create_sequences(data, time_steps=15):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i : i + time_steps])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

X, y = create_sequences(df["Close"].values, time_steps=15)

# Split data: 70% train, 10% validation, 20% test.
train_size = int(len(X) * 0.7)
val_size = int(len(X) * 0.1)
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val     = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test   = X[train_size+val_size:], y[train_size+val_size:]

# Reshape each sample to (time_steps, 1)
X_train = X_train.reshape(-1, 15, 1)
X_val   = X_val.reshape(-1, 15, 1)
X_test  = X_test.reshape(-1, 15, 1)

# ---------------------------
# 4. Build Multi-Scale Hybrid Model
# ---------------------------
from tensorflow.keras.layers import Multiply, Activation

def build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, 1))

    # *Branch 1: Full 15-Day Analysis (Long-Term Trends)*
    full_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(inputs)
    full_branch = Dense(64, activation='relu')(full_branch)
    long_term_return = Dense(1, activation='tanh')(full_branch)
    long_term_return = Lambda(lambda x: x * scale_factor)(long_term_return)

    # *Branch 2: Recent 7-Day Analysis (Mid-Term Trends)*
    recent_input = Lambda(lambda x: x[:, -7:, :])(inputs)  # Take the last 7 days
    recent_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(recent_input)
    recent_branch = Dense(64, activation='relu')(recent_branch)
    mid_term_return = Dense(1, activation='tanh')(recent_branch)
    mid_term_return = Lambda(lambda x: x * scale_factor)(mid_term_return)

    # *Branch 3: Very Recent 1-Day Analysis (Short-Term Trends)*
    very_recent_input = Lambda(lambda x: x[:, -1:, :])(inputs)  # Take the last day's price
    very_recent_branch = Flatten()(very_recent_input)
    very_recent_branch = Dense(32, activation='relu')(very_recent_branch)
    short_term_return = Dense(1, activation='tanh')(very_recent_branch)
    short_term_return = Lambda(lambda x: x * recent_scale)(short_term_return)

    # *Gating Mechanism (Forget Gate)*
    gate_7 = Dense(1, activation='sigmoid')(mid_term_return)  # Gate for 7-day impact
    gate_1 = Dense(1, activation='sigmoid')(short_term_return)  # Gate for 1-day impact

    gated_mid_term_return = Multiply()([mid_term_return, gate_7])  # 7-day result scaled by gate
    gated_short_term_return = Multiply()([short_term_return, gate_1])  # 1-day result scaled by gate

    # *Combine Long-Term, Gated Mid-Term, and Gated Short-Term Predictions*
    fused_returns = Add()([long_term_return, gated_mid_term_return, gated_short_term_return])
    fused_returns = Dense(32, activation='relu')(fused_returns)
    final_return = Dense(1, activation='tanh')(fused_returns)
    final_return = Lambda(lambda x: x * scale_factor)(final_return)

    # *Use the last day's price for the final output calculation*
    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, final_return])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
# Predictions on test set
##y_pred = model.predict(X_test)

# Inverse transform predictions and actual values
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

# Inverse transform predictions and actual values
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

# Calculate Metrics
r2 = r2_score(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mse = mean_squared_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Calculate the actual range of the test set
actual_range = np.max(y_test_inv) - np.min(y_test_inv)

# Compute Scaled Metrics
mae_scaled = mae / actual_range
mse_scaled = mse / (actual_range ** 2)
rmse_scaled = rmse / actual_range

# Explained Variance Score
evs = explained_variance_score(y_test_inv, y_pred_inv) 

# Print the results
print(f"📊 Final Results -")
print(f"  R²: {r2:.4f}")
print(f"  MAE: {mae:.4f}, MAE Scaled: {mae_scaled:.4f}")
print(f"  MSE: {mse:.4f}, MSE Scaled: {mse_scaled:.4f}")
print(f"  RMSE: {rmse:.4f}, RMSE Scaled: {rmse_scaled:.4f}")
print(f"  MAPE: {mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100:.2f}%")
print(f"  EVS: {evs:.4f}")

# OUR RANGE: NIFTY50 (15-3-2008 to 15-3-2024)

In [ ]:
vix = pd.read_csv("/kaggle/input/our-data-15-3-2008-to-15-3-2024/India VIX Historical Data (3).csv")
vix.drop("Vol.",axis=1,inplace=True)
vix["Change %"] = vix["Change %"].str.replace("%", "").astype(float)
vix.head(),vix.dtypes
data=pd.read_csv("/kaggle/input/our-data-15-3-2008-to-15-3-2024/Nifty 50 Historical Data (2).csv")
data=data[::-1]
data.reset_index(drop=True, inplace=True)
data.head()
data.nunique()

data.sort_index(axis=1,ascending=True)
data.rename(columns={'Price': 'Close'}, inplace=True)
df = data.copy()  # Ensure we don't modify the original dataset

# Convert financial columns to numeric (remove commas)
for col in ["Close", "Open", "High", "Low"]:
    df[col] = df[col].astype(str).str.replace(",", "").astype(float)

# Function to convert 'Vol.' column
def convert_volume(vol):
    if isinstance(vol, str):  # Ensure it's a string before replacing
        vol = vol.replace(",", "")  # Remove any thousand separators
        if "B" in vol:
            return float(vol.replace("B", "")) * 1_000_000_000
        elif "M" in vol:
            return float(vol.replace("M", "")) * 1_000_000
        elif "K" in vol:
            return float(vol.replace("K", "")) * 1_000
    return float(vol)  # Convert directly if already a number

df["Vol."] = df["Vol."].astype(str).apply(convert_volume)

# Convert 'Change %' column (remove '%' and convert to float)
df["Change %"] = df["Change %"].astype(str).str.replace("%", "").astype(float)

# Print final DataFrame
print(df.dtypes)
print(df)

# Assign back to 'data' (if needed)
data = df

data = pd.merge(data, vix, on='Date', how='inner')
data.drop(['Date'], axis=1, inplace=True)
data.head()

In [ ]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, explained_variance_score
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, Bidirectional, LSTM, Lambda, Concatenate, Add, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# ---------------------------
# 1. Set Seed for Reproducibility
# ---------------------------
def set_random_seed(seed=20):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.config.experimental.enable_op_determinism()

set_random_seed(20)

# ---------------------------
# 2. Load & Preprocess Data
# ---------------------------
# For demonstration, we use a basic univariate "Close" series.
# Ensure that your DataFrame 'data' contains at least a "Close" column.
df = data[["Close"]].copy()
scaler = MinMaxScaler()
df["Close"] = scaler.fit_transform(df)

# ---------------------------
# 3. Create 15-day Sequences
# ---------------------------
def create_sequences(data, time_steps=15):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i : i + time_steps])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

X, y = create_sequences(df["Close"].values, time_steps=15)

# Split data: 70% train, 10% validation, 20% test.
train_size = int(len(X) * 0.7)
val_size = int(len(X) * 0.1)
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val     = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test   = X[train_size+val_size:], y[train_size+val_size:]

# Reshape each sample to (time_steps, 1)
X_train = X_train.reshape(-1, 15, 1)
X_val   = X_val.reshape(-1, 15, 1)
X_test  = X_test.reshape(-1, 15, 1)

# ---------------------------
# 4. Build Multi-Scale Hybrid Model
# ---------------------------
from tensorflow.keras.layers import Multiply, Activation

def build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, 1))

    # *Branch 1: Full 15-Day Analysis (Long-Term Trends)*
    full_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(inputs)
    full_branch = Dense(64, activation='relu')(full_branch)
    long_term_return = Dense(1, activation='tanh')(full_branch)
    long_term_return = Lambda(lambda x: x * scale_factor)(long_term_return)

    # *Branch 2: Recent 7-Day Analysis (Mid-Term Trends)*
    recent_input = Lambda(lambda x: x[:, -7:, :])(inputs)  # Take the last 7 days
    recent_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(recent_input)
    recent_branch = Dense(64, activation='relu')(recent_branch)
    mid_term_return = Dense(1, activation='tanh')(recent_branch)
    mid_term_return = Lambda(lambda x: x * scale_factor)(mid_term_return)

    # *Branch 3: Very Recent 1-Day Analysis (Short-Term Trends)*
    very_recent_input = Lambda(lambda x: x[:, -1:, :])(inputs)  # Take the last day's price
    very_recent_branch = Flatten()(very_recent_input)
    very_recent_branch = Dense(32, activation='relu')(very_recent_branch)
    short_term_return = Dense(1, activation='tanh')(very_recent_branch)
    short_term_return = Lambda(lambda x: x * recent_scale)(short_term_return)

    # *Gating Mechanism (Forget Gate)*
    gate_7 = Dense(1, activation='sigmoid')(mid_term_return)  # Gate for 7-day impact
    gate_1 = Dense(1, activation='sigmoid')(short_term_return)  # Gate for 1-day impact

    gated_mid_term_return = Multiply()([mid_term_return, gate_7])  # 7-day result scaled by gate
    gated_short_term_return = Multiply()([short_term_return, gate_1])  # 1-day result scaled by gate

    # *Combine Long-Term, Gated Mid-Term, and Gated Short-Term Predictions*
    fused_returns = Add()([long_term_return, gated_mid_term_return, gated_short_term_return])
    fused_returns = Dense(32, activation='relu')(fused_returns)
    final_return = Dense(1, activation='tanh')(fused_returns)
    final_return = Lambda(lambda x: x * scale_factor)(final_return)

    # *Use the last day's price for the final output calculation*
    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, final_return])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
# Predictions on test set
##y_pred = model.predict(X_test)

# Inverse transform predictions and actual values
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

# Inverse transform predictions and actual values
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

# Calculate Metrics
r2 = r2_score(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mse = mean_squared_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Calculate the actual range of the test set
actual_range = np.max(y_test_inv) - np.min(y_test_inv)

# Compute Scaled Metrics
mae_scaled = mae / actual_range
mse_scaled = mse / (actual_range ** 2)
rmse_scaled = rmse / actual_range

# Explained Variance Score
evs = explained_variance_score(y_test_inv, y_pred_inv) 

# Print the results
print(f"📊 Final Results -")
print(f"  R²: {r2:.4f}")
print(f"  MAE: {mae:.4f}, MAE Scaled: {mae_scaled:.4f}")
print(f"  MSE: {mse:.4f}, MSE Scaled: {mse_scaled:.4f}")
print(f"  RMSE: {rmse:.4f}, RMSE Scaled: {rmse_scaled:.4f}")
print(f"  MAPE: {mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100:.2f}%")
print(f"  EVS: {evs:.4f}")

## DOING FOR SP500

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("/kaggle/input/personal-dateset-vix/SP 500 (2).csv")

# Ensure the required columns exist
if "Price" in df.columns and "Open" in df.columns:
    # Calculate the percentage change
    df["Change %"] = ((df["Price"] - df["Open"]) / df["Price"]) * 100

    # Save the updated CSV file
    # df.to_csv("SP 500 (2).csv", index=False)

    # Display first few rows
    print(df.head())
else:
    print("Error: CSV file must contain 'Price' and 'Open' columns.")


In [ ]:
import pandas as pd
import numpy as np
vix = pd.read_csv("/kaggle/input/personal-dateset-vix/SP 500 VIX.csv")
vix.drop("Vol.",axis=1,inplace=True)
def convert_volume(val):
    if pd.isna(val) or val == '-':  # Handle NaN or '-' cases
        return np.nan
    val = val.strip().replace(',', '')  # Remove spaces & commas
    if val[-1] == 'K': return float(val[:-1]) * 1e3
    if val[-1] == 'M': return float(val[:-1]) * 1e6
    if val[-1] == 'B': return float(val[:-1]) * 1e9
    return float(val)

#vix["Vol."] = vix["Vol."].apply(convert_volume)
vix.rename(columns={'Open':'Vopen','High':'Vhigh','Low':'Vlow','Change %':'Vchange%'}, inplace=True)
vix["Vchange%"] = vix["Vchange%"].str.replace("%", "").astype(float)
vix.head(),vix.dtypes

In [ ]:
data=df
data=data[::-1]
data.reset_index(drop=True, inplace=True)
data.head()
data.nunique()

data.sort_index(axis=1,ascending=True)
data.rename(columns={'Price': 'Close'}, inplace=True)
df = data.copy()  # Ensure we don't modify the original dataset

# Convert financial columns to numeric (remove commas)
for col in ["Close", "Open", "High", "Low"]:
    df[col] = df[col].astype(str).str.replace(",", "").astype(float)

# Function to convert 'Vol.' column
#df.drop("Vol.",axis=1,inplace=True)

# Convert 'Change %' column (remove '%' and convert to float)
df["Change %"] = df["Change %"].astype(str).str.replace("%", "").astype(float)

# Print final DataFrame
print(df.dtypes)
print(df)

# Assign back to 'data' (if needed)
data = df
data['Date'] = pd.to_datetime(data['Date'], format="%d-%m-%Y")

# For 'vix', the date format is "MM/DD/YYYY"
vix['Date'] = pd.to_datetime(vix['Date'], format="%m/%d/%Y")
data = pd.merge(data, vix, on='Date', how='inner')
row=data[data["Vol."].isna()]
data.drop(['Date'], axis=1, inplace=True)
data.head()

In [ ]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, explained_variance_score
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, Bidirectional, LSTM, Lambda, Concatenate, Add, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# ---------------------------
# 1. Set Seed for Reproducibility
# ---------------------------
def set_random_seed(seed=20):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.config.experimental.enable_op_determinism()

set_random_seed(20)

# ---------------------------
# 2. Load & Preprocess Data
# ---------------------------
# For demonstration, we use a basic univariate "Close" series.
# Ensure that your DataFrame 'data' contains at least a "Close" column.
df = data[["Close"]].copy()
scaler = MinMaxScaler()
df["Close"] = scaler.fit_transform(df)

# ---------------------------
# 3. Create 15-day Sequences
# ---------------------------
def create_sequences(data, time_steps=15):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i : i + time_steps])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

X, y = create_sequences(df["Close"].values, time_steps=15)

# Split data: 70% train, 10% validation, 20% test.
train_size = int(len(X) * 0.7)
val_size = int(len(X) * 0.1)
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val     = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test   = X[train_size+val_size:], y[train_size+val_size:]

# Reshape each sample to (time_steps, 1)
X_train = X_train.reshape(-1, 15, 1)
X_val   = X_val.reshape(-1, 15, 1)
X_test  = X_test.reshape(-1, 15, 1)

# ---------------------------
# 4. Build Multi-Scale Hybrid Model
# ---------------------------
from tensorflow.keras.layers import Multiply, Activation

def build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1):
    inputs = Input(shape=(seq_length, 1))

    # *Branch 1: Full 15-Day Analysis (Long-Term Trends)*
    full_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(inputs)
    full_branch = Dense(64, activation='relu')(full_branch)
    long_term_return = Dense(1, activation='tanh')(full_branch)
    long_term_return = Lambda(lambda x: x * scale_factor)(long_term_return)

    # *Branch 2: Recent 7-Day Analysis (Mid-Term Trends)*
    recent_input = Lambda(lambda x: x[:, -7:, :])(inputs)  # Take the last 7 days
    recent_branch = Bidirectional(LSTM(128, return_sequences=False, dropout=0.2))(recent_input)
    recent_branch = Dense(64, activation='relu')(recent_branch)
    mid_term_return = Dense(1, activation='tanh')(recent_branch)
    mid_term_return = Lambda(lambda x: x * scale_factor)(mid_term_return)

    # *Branch 3: Very Recent 1-Day Analysis (Short-Term Trends)*
    very_recent_input = Lambda(lambda x: x[:, -1:, :])(inputs)  # Take the last day's price
    very_recent_branch = Flatten()(very_recent_input)
    very_recent_branch = Dense(32, activation='relu')(very_recent_branch)
    short_term_return = Dense(1, activation='tanh')(very_recent_branch)
    short_term_return = Lambda(lambda x: x * recent_scale)(short_term_return)

    # *Gating Mechanism (Forget Gate)*
    gate_7 = Dense(1, activation='sigmoid')(mid_term_return)  # Gate for 7-day impact
    gate_1 = Dense(1, activation='sigmoid')(short_term_return)  # Gate for 1-day impact

    gated_mid_term_return = Multiply()([mid_term_return, gate_7])  # 7-day result scaled by gate
    gated_short_term_return = Multiply()([short_term_return, gate_1])  # 1-day result scaled by gate

    # *Combine Long-Term, Gated Mid-Term, and Gated Short-Term Predictions*
    fused_returns = Add()([long_term_return, gated_mid_term_return, gated_short_term_return])
    fused_returns = Dense(32, activation='relu')(fused_returns)
    final_return = Dense(1, activation='tanh')(fused_returns)
    final_return = Lambda(lambda x: x * scale_factor)(final_return)

    # *Use the last day's price for the final output calculation*
    last_day = Lambda(lambda x: tf.expand_dims(x[:, -1, 0], axis=-1))(inputs)
    final_output = Lambda(lambda inputs: inputs[0] * (1 + inputs[1]))([last_day, final_return])

    model = Model(inputs, final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    return model

# *Build the Modified Model with Gating*
# model_gated = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
# model_gated.summary()

# *Build the Modified Model with Attention-Based Residuals*
model_attention = build_gated_multi_scale_model(seq_length=15, scale_factor=0.2, recent_scale=0.1)
model_attention.summary()
early_stop = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
history = model_attention.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=8, callbacks=[early_stop], verbose=1)

# ---------------------------
# 7. Evaluate the Model
# ---------------------------
y_pred = model_attention.predict(X_test)
# Predictions on test set
##y_pred = model.predict(X_test)

# Inverse transform predictions and actual values
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

# Inverse transform predictions and actual values
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

# Calculate Metrics
r2 = r2_score(y_test_inv, y_pred_inv)
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mse = mean_squared_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mse)  # Root Mean Squared Error

# Calculate the actual range of the test set
actual_range = np.max(y_test_inv) - np.min(y_test_inv)

# Compute Scaled Metrics
mae_scaled = mae / actual_range
mse_scaled = mse / (actual_range ** 2)
rmse_scaled = rmse / actual_range

# Explained Variance Score
evs = explained_variance_score(y_test_inv, y_pred_inv) 

# Print the results
print(f"📊 Final Results -")
print(f"  R²: {r2:.4f}")
print(f"  MAE: {mae:.4f}, MAE Scaled: {mae_scaled:.4f}")
print(f"  MSE: {mse:.4f}, MSE Scaled: {mse_scaled:.4f}")
print(f"  RMSE: {rmse:.4f}, RMSE Scaled: {rmse_scaled:.4f}")
print(f"  MAPE: {mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100:.2f}%")
print(f"  EVS: {evs:.4f}")

In [ ]:


plt.figure(figsize=(10,5))
plt.plot(y_test_inv, label='Actual')
plt.plot(y_pred_inv, label='Predicted')
plt.xlabel("Time")
plt.ylabel("Close Price")
plt.legend()
plt.show()